## 概要

Open-Sora 2.0は、3000万円で実現可能な商用レベルの動画生成モデル

訓練コストは、同等のモデル（MovieGenやStep-Video-T2V）よりも5~10倍低い

人の評価とVBenchのスコアでは、Huyyuan VideoやRunway Gen-3 Alphaに匹敵:

![](image/fig1.png)

- Visual Quality: 視覚品質
- Prompt Following: 指示追従性
- Motion Quality: 動きの品質

## データ

目的は、学習の進捗に合わせたデータピラミッド（hierarchical data pyramid）の構築

様々な種類のデータを検出可能ななフィルタを開発

学習の進捗に応じて、フィルタリングの強度を高め、純度と品質の高い小さいサブセットで訓練

データフィルタリングの全体像:

![](image/fig2.png)

- 紫: 生の動画の前処理
    1. フィルタリング
        - 破損したファイルの除去
        - 極端な動画の除去
            - 再生時間が2秒未満
            - 1画像あたりのデータ量（Bit per pixel）が0.02未満
            - フレームレート（fps）が16未満
            - アスペクト比が範囲外（1/3, 3）
            - 特定の低品質なエンコード設定（Constrained Baseline profile）
    2. 連続した映像を検出し、短いクリップに分割
        - FFmpegのlibavfilterを使用し、シーンスコア（フレーム間の視覚差分）を計算
    3. 動画のフォーマット
        - フレームレートは30fps以下
        - 長辺は1080px以下
        - コーデック（圧縮形式）はH.264
        - 動画の黒帯を削除
        - 8秒を超えるショットは、8秒のクリップに分割し、2秒未満は破棄
- 青: クリップ動画のスコアフィルタリング
    - 美的スコアでのフィルタリング
        - [CLIPとMLP][1]で美的スコアを予測
            - 最初・中間・最終フレームを抽出し、スコアを計算し、平均
    - 鮮明さの低い動画のフィルタリング
        - OpenCVのラプラシアン演算子で画像の分散が低い（ぼやけている）動画を除去
    - 手ブレの多い動画のフィルタリング
        - PySceneDetectを使用して、フレーム間の変化が大きい動画を除去
    - 重複クリップのフィルタリング
- 緑: 256pxの低解像度動画
    - 多くの文章が含まれる動画のフィルタリング
        - PaddleOCRを使用して、テキストのバウンディングボックスを検出
        - 信頼度スコアが0.7を超えるボックスの総面積が多い動画を除去
    - モーションスコアでのフィルタリング
        - libavfilterのVMAFを使用して動画の動きの激しさを測定
        - モーションスコアが極端に低い・高い動画を除去
- 黄色: 768pxの高解像度動画

[1]: https://github.com/christophschuhmann/improved-aesthetic-predictor

動画のキャプションを作成するために、視覚言語モデルを使用:

- 256px動画には、LLaVA-Video
- 778px動画には、Qwen 2.5 Max（ハルシネーションが少ないプロプライエタリモデル）

キャプション生成のプロンプトの構成:

- 主な被写体
- 被写体の動き
- 背景や環境
- 証明条件や雰囲気
- カメラワーク
- リアル・シネマティック・3D・アニメなどの動画のスタイル

生成時に動きの強度を調整可能にするため、キャプションの最後にモーションスコアを追記

フィルタリング後のデータの統計:

![](image/fig3.png)

- 美的スコアは4.5~5.5で中程度
- 動画の長さは2~8秒で、半分近くが6~8秒
- アスペクト比は、大部分が0.5~0.75（16:9の横長動画）
- キャプションの70%は75単語を超えていて情報量が多い

キャプションのワードクラウド:

![](image/fig4.png)

- 背景や照明条件も含まれていて、被写体は人物が多い

## アーキテクチャ

### 3次元オートエンコーダ

[Hunyuan Video VAE][1]のアーキテクチャを効率化したVideo DC-AEを開発

DC-AEは、[Deep Compression Autoencoder][2]の略

圧縮率は、$4\times 32\times 32$（時間は $\frac{1}{4}$、縦と横は$\frac{1}{32}$に圧縮）

オートエンコーダーの学習データは32フレーム、256pxのため、潜在表現は$8\times 8\times 8$

[1]: https://arxiv.org/abs/2412.03603
[2]: https://arxiv.org/abs/2410.10733

Video DC-AEのアーキテクチャ:

![](image/fig5.png)

- エンコーダ
    - 3層のResBlockと3層のEfficientViT Blockで構成される
    - 最初の5つのブロックは、ダウンサンプリング用
    - 学習を可能にするため、ダウンサンプルブロックには、残差接続が導入
    - 残差接続はピクセルアンシャッフリングを使用（Space&Time->Channel）
- デコーダ
    - 3層のEfficientViT Blockと3層のResBlockで構成される
    - 最後の5つのブロックは、アップサンプリング用
    - 学習を可能にするため、アップサンプルブロックには、残差接続が導入
    - 残差接続はピクセルシャッフリングを使用（Channel->Space&Time）

Video DC-AEをスクラッチから学習し、再構成品質を評価:

![](image/table1.png)

- LPIPS: 人間の知覚に近い画質評価指標

## DiTアーキテクチャ

離れたフレームや画素同士の関係を効果的に捉えるフルアテンションを採用

動画はVideo DC-AEで圧縮後、パッチサイズ1（=パッチ化無し）でフラット化

Hunyuan Videoのオートエンコーダーを使用する場合は、パッチサイズ2が必要

FLUXの[MMDiT][1]を参考に、デュアルストリームとシングルストリームからなる構造を採用:

![](image/fig6.png)

[1]: https://github.com/black-forest-labs/flux

- デュアルストリームブロックで、動画とテキストが別々に特徴抽出される
- シングルストリームブロックで、特徴を統合する

空間と時間情報を捉えるために、3D RoPEを採用

テキストノエンコードは、2つの事前学習済みモデルを採用:

- T5-XXL: 複雑なテキストの意味を捉える
- CLIP-Large: テキストと視覚概念の整合性を捉える（=指示追従性を高める）

![](image/table2.png)

## 実装

- image.py: 画像のみで学習。
- stage1.py: 256px解像度の動画で学習。
- stage2.py: 768px解像度の動画で学習（シーケンス並列化を使用、デフォルトは4）。
- stage1_i2v.py: 256px解像度でT2V（テキストから動画）とI2V（画像から動画）を学習。
- stage2_i2v.py: 768px解像度でT2VとI2Vを学習。

## 環境構築

In [ ]:
%load_ext autoreload
%autoreload 2
%cd /workspaces/open-sora/Open-Sora

In [ ]:
import logging
import os
import platform
import subprocess

if os.path.exists("debug.log"):
    os.remove("debug.log")

def custom_format(record):
    match record.levelno:
        case logging.DEBUG:
            level = "🟦"
        case logging.INFO:
            level = "🟩"
        case logging.WARNING:
            level = "🟨"
        case logging.ERROR:
            level = "🟥"
        case logging.CRITICAL:
            level = "🛑"
    return f"{level} {record.getMessage()}"

logger = logging.getLogger()

for handler in logger.handlers:
    logger.removeHandler(handler)

formatter = logging.Formatter()
formatter.format = custom_format

file_handler = logging.FileHandler("debug.log")
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)

stream_handler = logging.StreamHandler()
stream_handler.setFormatter(formatter)
logger.addHandler(stream_handler)
logger.setLevel(logging.DEBUG)

PYTHON_VERSION = platform.python_version()
logger.info(f"Python {PYTHON_VERSION}")

NVIDIA_SMI = subprocess.run("nvidia-smi", capture_output=True, text=True).stdout
logger.info(f"{NVIDIA_SMI}")

In [ ]:
if False:
    %pip install torch==2.5.1 torchvision==0.20.1 torchaudio==2.5.1 --index-url https://download.pytorch.org/whl/cu124

    %pip install \
        accelerate \
        av \
        colossalai \
        ftfy \
        liger-kernel \
        omegaconf \
        mmengine \
        openai \
        pandas \
        pandarallel \
        pyarrow \
        tensorboard \
        wandb \
        --extra-index-url https://download.pytorch.org/whl/cu124

    %pip install flash-attn --no-build-isolation

    %pip install -e . --no-deps

In [ ]:
# %pip install -qU av==13.1.0

In [ ]:
from dataclasses import dataclass, field
from einops import rearrange
from flash_attn import flash_attn_func as flash_attn_func_v2
from functools import partial
from huggingface_hub import PyTorchModelHubMixin
from inspect import signature
from liger_kernel.ops.rms_norm import LigerRMSNormFunction
from liger_kernel.ops.rope import LigerRopeFunction
from omegaconf import MISSING, OmegaConf
from torch import Tensor, nn
from torch.nn.modules.batchnorm import _BatchNorm
from typing import Any, Callable, Optional, Union, Tuple
import diffusers
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import argparse
from mmengine.config import Config
import ast

try:
    from flash_attn_interface import flash_attn_func as flash_attn_func_v3
    SUPPORT_FA3 = True
except:
    SUPPORT_FA3 = False

## ダウンロード

In [ ]:
# https://huggingface.co/hpcai-tech/Open-Sora-v2-Video-DC-AE
# F32T4C128_AE.safetensors 919MB
# Open_Sora_v2_Video_DC_AE.safetensors 23.8GB

if False:
    !huggingface-cli download hpcai-tech/Open-Sora-v2-Video-DC-AE --local-dir ./ckpts

## CLIパーサ

In [ ]:
def parse_args(args) -> tuple[str, argparse.Namespace]:
    """
    This function parses the command line arguments.

    Returns:
        tuple[str, argparse.Namespace]: The path to the configuration file and the command line arguments.
    """
    parser = argparse.ArgumentParser()
    parser.add_argument("config", type=str, help="model config file path")
    args, unknown_args = parser.parse_known_args(args)
    return args.config, unknown_args

In [ ]:
def read_config(config_path: str) -> Config:
    """
    This function reads the configuration file.

    Args:
        config_path (str): The path to the configuration file.

    Returns:
        Config: The configuration object.
    """
    cfg = Config.fromfile(config_path)
    return cfg

In [ ]:
def auto_convert(value: str) -> int | float | bool | list | dict | None:
    """
    Automatically convert a string to the appropriate Python data type,
    including int, float, bool, list, dict, etc.

    Args:
        value (str): The string to convert.

    Returns:
        int, float, bool, list |  dict: The converted value.
    """
    # Handle empty string
    if value == "":
        return value

    # Handle None
    if value.lower() == "none":
        return None

    # Handle boolean values
    lower_value = value.lower()
    if lower_value == "true":
        return True
    elif lower_value == "false":
        return False

    # Try to convert the string to an integer or float
    try:
        # Try converting to an integer
        return int(value)
    except ValueError:
        pass

    try:
        # Try converting to a float
        return float(value)
    except ValueError:
        pass

    # Try to convert the string to a list, dict, tuple, etc.
    try:
        return ast.literal_eval(value)
    except (ValueError, SyntaxError):
        pass

    # If all attempts fail, return the original string
    return value


In [ ]:
def merge_args(cfg: Config, args: argparse.Namespace) -> Config:
    """
    This function merges the configuration file and command line arguments.

    Args:
        cfg (Config): The configuration object.
        args (argparse.Namespace): The command line arguments.

    Returns:
        Config: The configuration object.
    """
    for k, v in zip(args[::2], args[1::2]):
        assert k.startswith("--"), f"Invalid argument: {k}"
        k = k[2:].replace("-", "_")
        k_split = k.split(".")
        target = cfg
        for key in k_split[:-1]:
            assert key in cfg, f"Key {key} not found in config"
            target = target[key]
        if v.lower() == "none":
            v = None
        elif k in target:
            v_type = type(target[k])
            if v_type == bool:
                v = auto_convert(v)
            else:
                v = type(target[k])(v)
        else:
            v = auto_convert(v)
        target[k_split[-1]] = v
    return cfg

In [ ]:
def parse_configs(args) -> Config:
    """
    This function parses the configuration file and command line arguments.

    Returns:
        Config: The configuration object.
    """
    config, args = parse_args(args)
    cfg = read_config(config)
    cfg = merge_args(cfg, args)
    cfg.config_path = config

    # hard-coded for spatial compression
    if cfg.get("ae_spatial_compression", None) is not None:
        os.environ["AE_SPATIAL_COMPRESSION"] = str(cfg.ae_spatial_compression)

    return cfg

### オードエンコーダの推論

In [ ]:
# torchrun --nproc_per_node 1 --standalone scripts/vae/inference.py configs/vae/inference/video_dc_ae.py --save-dir samples/dcae

parse_configs([
    "configs/vae/inference/video_dc_ae.py",
    "--save-dir",
    "samples/dcae",
])

### DC-AE用の動画生成

In [ ]:
# torchrun --nproc_per_node 1 --standalone scripts/diffusion/inference.py configs/diffusion/inference/high_compression.py --prompt "The story of a robot's life in a cyberpunk setting." 

parse_configs([
    "configs/diffusion/inference/high_compression.py",
    "--prompt",
    "The story of a robot's life in a cyberpunk setting.",
])

### hunyuanVideoAE用の動画生成

In [ ]:
# torchrun --nproc_per_node 1 --standalone scripts/diffusion/inference.py configs/diffusion/inference/256px.py --prompt "raining, sea"

parse_configs([
    "configs/diffusion/inference/256px.py",
    "--prompt",
    "raining, sea",
])

In [ ]:
# torchrun --nproc_per_node 8 --standalone scripts/diffusion/inference.py configs/diffusion/inference/768px.py --prompt "raining, sea"

parse_configs([
    "configs/diffusion/inference/768px.py",
    "--prompt",
    "raining, sea",
])

In [ ]:
# Text-to-Video 256x256

# torchrun --nproc_per_node 1 --standalone scripts/diffusion/inference.py configs/diffusion/inference/t2i2v_256px.py --save-dir samples --prompt "raining, sea"

parse_configs([
    "configs/diffusion/inference/t2i2v_256px.py",
    "--save-dir",
    "samples",
    "--prompt",
    "raining, sea",
])

In [ ]:
# Text-to-Video 256x256 from CSV

# torchrun --nproc_per_node 1 --standalone scripts/diffusion/inference.py configs/diffusion/inference/t2i2v_256px.py --save-dir samples --dataset.data-path assets/texts/example.csv

parse_configs([
    "configs/diffusion/inference/t2i2v_256px.py",
    "--save-dir",
    "samples",
    "--dataset.data-path",
    "assets/texts/example.csv",
])

In [ ]:
# Text-to-Video 768x768 

# torchrun --nproc_per_node 1 --standalone scripts/diffusion/inference.py configs/diffusion/inference/t2i2v_768px.py --save-dir samples --prompt "raining, sea"

parse_configs([
    "configs/diffusion/inference/t2i2v_768px.py",
    "--save-dir",
    "samples",
    "--prompt",
    "raining, sea",
])

In [ ]:
# Text-to-Video 768x768 multi-GPU

# torchrun --nproc_per_node 8 --standalone scripts/diffusion/inference.py configs/diffusion/inference/t2i2v_768px.py --save-dir samples --prompt "raining, sea"

parse_configs([
    "configs/diffusion/inference/t2i2v_768px.py",
    "--save-dir",
    "samples",
    "--prompt",
    "raining, sea",
])

### 画像から動画生成

In [ ]:
# torchrun --nproc_per_node 1 --standalone scripts/diffusion/inference.py configs/diffusion/inference/256px.py --cond_type i2v_head --prompt "A plump pig wallows in a muddy pond on a rustic farm, its pink snout poking out as it snorts contentedly. The camera captures the pig's playful splashes, sending ripples through the water under the midday sun. Wooden fences and a red barn stand in the background, framed by rolling green hills. The pig's muddy coat glistens in the sunlight, showcasing the simple pleasures of its carefree life." --ref assets/texts/i2v.png

parse_configs([
    "configs/diffusion/inference/256px.py",
    "--cond_type",
    "i2v_head",
    "--prompt",
    "A plump pig wallows in a muddy pond on a rustic farm, its pink snout poking out as it snorts contentedly. The camera captures the pig's playful splashes, sending ripples through the water under the midday sun. Wooden fences and a red barn stand in the background, framed by rolling green hills. The pig's muddy coat glistens in the sunlight, showcasing the simple pleasures of its carefree life.",
    "--ref",
    "assets/texts/i2v.png",
])

In [ ]:
# 256px with csv

# torchrun --nproc_per_node 1 --standalone scripts/diffusion/inference.py configs/diffusion/inference/256px.py --cond_type i2v_head --dataset.data-path assets/texts/i2v.csv

parse_configs([
    "configs/diffusion/inference/256px.py",
    "--cond_type",
    "i2v_head",
    "--dataset.data-path",
    "assets/texts/i2v.csv",
])

In [ ]:
# Multi-GPU 768px

# torchrun --nproc_per_node 8 --standalone scripts/diffusion/inference.py configs/diffusion/inference/768px.py --cond_type i2v_head --dataset.data-path assets/texts/i2v.csv

parse_configs([
    "configs/diffusion/inference/768px.py",
    "--cond_type",
    "i2v_head",
    "--dataset.data-path",
    "assets/texts/i2v.csv",
])

### モーションスコアを指定した動画生成

In [ ]:
# torchrun --nproc_per_node 1 --standalone scripts/diffusion/inference.py configs/diffusion/inference/t2i2v_256px.py --save-dir samples --prompt "raining, sea" --motion-score 4

parse_configs([
    "configs/diffusion/inference/t2i2v_256px.py",
    "--save-dir",
    "samples",
    "--prompt",
    "raining, sea",
    "--motion-score",
    "4",
])

In [ ]:
# 動画のモーションスコアを評価
# OpenAI API Keyが必要

# torchrun --nproc_per_node 1 --standalone scripts/diffusion/inference.py configs/diffusion/inference/t2i2v_256px.py --save-dir samples --prompt "raining, sea" --motion-score dynamic

parse_configs([
    "configs/diffusion/inference/t2i2v_256px.py",
    "--save-dir",
    "samples",
    "--prompt",
    "raining, sea",
    "--motion-score",
    "dynamic",
])

### プロンプトの洗練

In [ ]:
# export OPENAI_API_KEY=sk-xxxx
# torchrun --nproc_per_node 1 --standalone scripts/diffusion/inference.py configs/diffusion/inference/t2i2v_256px.py --save-dir samples --prompt "raining, sea" --refine-prompt True

parse_configs([
    "configs/diffusion/inference/t2i2v_256px.py",
    "--save-dir",
    "samples",
    "--prompt",
    "raining, sea",
    "--refine-prompt",
    "True",
])

### シード固定

In [ ]:
# torchrun --nproc_per_node 1 --standalone scripts/diffusion/inference.py configs/diffusion/inference/t2i2v_256px.py --save-dir samples --prompt "raining, sea" --sampling_option.seed 42 --seed 42

parse_configs([
    "configs/diffusion/inference/t2i2v_256px.py",
    "--save-dir",
    "samples",
    "--prompt",
    "raining, sea",
    "--sampling_option.seed",
    "42",
    "--seed",
    "42",
])

### 訓練設定

In [ ]:
# DC-AE訓練

# https://github.com/hpcaitech/Open-Sora/blob/main/docs/hcae.md

# torchrun --nproc_per_node 8 scripts/vae/train.py configs/vae/train/video_dc_ae.py

parse_configs([
    "configs/vae/train/video_dc_ae.py",
])

In [ ]:
# DC-AE-DISC訓練

# torchrun --nproc_per_node 8 scripts/vae/train.py configs/vae/train/video_dc_ae_disc.py --model.from_pretrained <model_ckpt>

# args_2 = [
#     "configs/vae/train/video_dc_ae_disc.py",
#     "--model.from_pretrained",
#     "<model_ckpt>"
# ]

parse_configs([
    "configs/vae/train/video_dc_ae_disc.py",
    "--model.from_pretrained",
    "<model_ckpt>"
])

In [ ]:
# Diffusion訓練

# torchrun --nproc_per_node 8 scripts/diffusion/train.py configs/diffusion/train/stage1.py --dataset.data-path datasets/pexels_45k_necessary.csv

parse_configs([
    "configs/diffusion/train/stage1.py",
    "--dataset.data-path",
    "datasets/pexels_45k_necessary.csv"
])

## DC-AE

### ユーティリティ関数

In [ ]:
def ceil_to_divisible(n: int, dividend: int) -> int:
    return math.ceil(dividend / (dividend // n))

In [ ]:
def val2list(x: Union[list, tuple, Any], repeat_time=1) -> list:
    """
    値をリストに変換する

    Args:
        x (Union[list, tuple, Any]): 変換する値
        repeat_time (int, optional): xがリストやタプルでない場合の繰り返し回数
    Returns:
        list: 変換後のリスト
    """

    # リストかタプルの場合
    if isinstance(x, (list, tuple)):
        return list(x)

    # それ以外の場合
    return [x for _ in range(repeat_time)]

# 検証
val2list([1,2,3]), val2list((4,5)), val2list(5, repeat_time=3)

In [ ]:
def val2tuple(x: Union[list, tuple, Any], min_len: int = 1, idx_repeat: int = -1) -> tuple:
    """
    値をタプルに変換する

    Args:
        x (Union[list, tuple, Any]): 変換する値
        min_len (int, optional): タプルの最小長さ
        idx_repeat (int, optional): 繰り返し挿入するインデックス
    Returns:
        tuple: 変換後のタプル
    """

    # 値をリストに変換
    x = val2list(x)

    # 必要に応じて要素を繰り返し
    if len(x) > 0:
        x[idx_repeat:idx_repeat] = [
            x[idx_repeat] for _ in range(min_len - len(x))
        ]

    return tuple(x)

# 検証
val2tuple((True, False), 2)

In [ ]:
def build_kwargs_from_config(config: dict, target_func: Callable) -> dict[str, Any]:
    """
    設定から関数のキーワード引数を構築する

    Args:
        config (dict): 設定辞書
        target_func (Callable): 対象の関数
    Returns:
        dict[str, Any]: 関数に渡すキーワード引数
    """
    logger.info(f"build_kwargs_from_config {config=} {target_func=}")

    valid_keys = list(signature(target_func).parameters)
    kwargs = {}
    for key in config:
        if key in valid_keys:
            kwargs[key] = config[key]
    return kwargs

In [ ]:
def get_same_padding(kernel_size: Union[int, tuple[int, ...]]) -> Union[int, tuple[int, ...]]:
    """
    カーネルサイズに基づいて同じパディングを計算する
    ConvLayer, LiteMLAで使用

    Args:
        kernel_size (Union[int, tuple[int, ...]]): カーネルサイズ
    Returns:
        Union[int, tuple[int, ...]]: 同じパディングサイズ
    """
    logger.info(f"get_same_padding {kernel_size=}")

    # False
    if isinstance(kernel_size, tuple):
        result =  tuple([get_same_padding(ks) for ks in kernel_size])
        loggger.debug(f"get_same_padding result: {result}")
        return result

    # True
    else:
        assert kernel_size % 2 > 0, "kernel size should be odd number"

        # 5 // 2 = 2
        result = kernel_size // 2
        logger.debug(f"get_same_padding result: {result}")
        return result

In [ ]:
def auto_grad_checkpoint(module, *args, **kwargs):
    """
    自動勾配チェックポイント関数
    勾配チェックポイントが有効な場合、チェックポイントを使用してモジュールを実行する
    そうでない場合、通常のフォワードパスを実行する

    Args:
        module: nn.Moduleまたはnn.ModuleのIterable
        *args: モジュールへの引数
        **kwargs: モジュールへのキーワード引数
    Returns:
        モジュールの出力
    """
    logger.info(f"auto_grad_checkpoint {len(args)=} {args[0].shape=} {kwargs=}")

    # False
    if getattr(module, "grad_checkpointing", False):
        if not isinstance(module, Iterable):
            return checkpoint(module, *args, use_reentrant=True, **kwargs)

        gc_step = module[0].grad_checkpointing_step
        logger.debug(f"gc_step: {gc_step}")

        return checkpoint_sequential(module, gc_step, *args, use_reentrant=False, **kwargs)

    return module(*args, **kwargs)

### モジュールリスト

In [ ]:
class OpSequential(nn.Module):
    """
    複数の操作を順次適用するモジュール
    """
    def __init__(self, op_list: list[Optional[nn.Module]]):
        logger.info(f"OpSequential init with {op_list=}")
        super().__init__()
        valid_op_list = []
        for op in op_list:
            if op is not None:
                valid_op_list.append(op)
        self.op_list = nn.ModuleList(valid_op_list)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        logger.info(f"OpSequential forward {x.shape=}")
        for op in self.op_list:
            x = op(x)
        return x

### 恒等層

In [ ]:
class IdentityLayer(nn.Module):
    """
    恒等層
    残差接続のショートカットパス用
    build_block, EfficientViTBlockで使用
    """
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        logger.info(f"IdentityLayer forward {x.shape=}")
        return x

### 正規化層

In [ ]:
class RMSNorm2d(nn.Module):
    def __init__(
        self, num_features: int, eps: float = 1e-5, elementwise_affine: bool = True, bias: bool = True
    ) -> None:
        """
        2次元RMS正規化層

        Args:
            num_features (int): 正規化する特徴量の数
            eps (float, optional): 数値安定性のための小さな値
            elementwise_affine (bool, optional): 要素ごとのアフィン変換を使用するか
            bias (bool, optional): バイアス項を使用するか
        """

        logger.info(f"RMSNorm2d init with {num_features=}, {eps=}, {elementwise_affine=}, {bias=}")
        super().__init__()

        self.num_features = num_features

        self.eps = eps

        self.elementwise_affine = elementwise_affine

        if self.elementwise_affine:
            self.weight = torch.nn.parameter.Parameter(
                torch.empty(self.num_features)
            )

            if bias:
                self.bias = torch.nn.parameter.Parameter(
                    torch.empty(self.num_features)
                )
            else:
                self.register_parameter("bias", None)
        else:
            self.register_parameter("weight", None)
            self.register_parameter("bias", None)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = (x / torch.sqrt(torch.square(x.float()).mean(dim=1, keepdim=True) + self.eps)).to(x.dtype)
        if self.elementwise_affine:
            x = x * self.weight.view(1, -1, 1, 1) + self.bias.view(1, -1, 1, 1)
        return x

In [ ]:
class RMSNorm3d(RMSNorm2d):
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        3次元RMS正規化層のフォワードパス
        Args:
            x (torch.Tensor): 入力テンソル (B, C, D, H, W)
        Returns:
            torch.Tensor: 正規化された出力テンソル
        """
        logger.info(f"RMSNorm3d forward {x.shape=}")

        x = (
            x / torch.sqrt(torch.square(x.float()).mean(dim=1, keepdim=True) +
            self.eps)
        ).to(x.dtype)

        if self.elementwise_affine:
            x = x * self.weight.view(1, -1, 1, 1, 1) + self.bias.view(1, -1, 1, 1, 1)

        return x

In [ ]:
# register normalization function here
REGISTERED_NORM_DICT: dict[str, type] = {
    # "bn2d": nn.BatchNorm2d,
    # "ln": nn.LayerNorm,
    # "ln2d": LayerNorm2d,
    # "rms2d": RMSNorm2d,
    "rms3d": RMSNorm3d,
}

In [ ]:
def build_norm(name="bn2d", num_features=None, **kwargs) -> Optional[nn.Module]:
    """
    正規化層を構築する
    RMSNorm3dのみ使用

    Args:
        name (str, optional): 正規化層の名前
        num_features (int, optional): 正規化層の特徴数
        **kwargs: 正規化層の追加引数
    Returns:
        Optional[nn.Module]: 構築された正規化層、またはNone
    """
    logger.info(f"build_norm {name=} {num_features=} {kwargs=}")

    # False
    if name in ["ln", "ln2d"]:
        kwargs["normalized_shape"] = num_features
    else:
        kwargs["num_features"] = num_features

    if name in REGISTERED_NORM_DICT:
        norm_cls = REGISTERED_NORM_DICT[name]
        args = build_kwargs_from_config(kwargs, norm_cls)
        return norm_cls(**args)
    else:
        return None

### 活性化関数

In [ ]:
REGISTERED_ACT_DICT: dict[str, type] = {
    "relu": nn.ReLU,
    # "relu6": nn.ReLU6,
    # "hswish": nn.Hardswish,
    "silu": nn.SiLU,
    # "gelu": partial(nn.GELU, approximate="tanh"),
}

In [ ]:
def build_act(name: str, **kwargs) -> Optional[nn.Module]:
    """
    活性化関数を構築する
    ReLUかSiLuのみ使用

    Args:
        name (str): 活性化関数の名前
        **kwargs: 活性化関数の追加引数
    Returns:
        Optional[nn.Module]: 構築された活性化関数、またはNone
    """
    logger.info(f"build_act {name=} {kwargs=}")
    if name in REGISTERED_ACT_DICT:
        act_cls = REGISTERED_ACT_DICT[name]
        args = build_kwargs_from_config(kwargs, act_cls)
        return act_cls(**args)
    else:
        return None

### 畳み込み層

In [ ]:
class ChannelChunkConv3d(nn.Conv3d):
    """
    チャンネルをチャンクに分割してConv3dを実行するクラス
    メモリ不足(OOM)を回避するために、チャンネルを分割して少しずつ計算し、結果を結合する
    ConvLayerとLiteMLAで使用
    合計170回使用
    """

    # 演算上の要素数制限
    CONV3D_NUMEL_LIMIT = 2**31

    def _get_output_numel(self, input_shape: torch.Size) -> int:
        """
        出力テンソルの演算上の要素数を計算する

        Args:
            input_shape (torch.Size): 入力テンソルの形状
        Returns:
            int: 出力テンソルの演算上の要素数
        """
        logger.info(f"ChannelChunkConv3d._get_output_numel {input_shape=}")

        numel = self.out_channels
        logger.debug(f"ChannelChunkConv3d._get_output_numel initial {numel=}")

        # True
        if len(input_shape) == 5:
            numel *= input_shape[0]

        for i, d in enumerate(input_shape[-3:]):
            d_out = math.floor(
                (d + 2 * self.padding[i] - self.dilation[i] * (self.kernel_size[i] - 1) - 1) / self.stride[i] + 1
            )
            numel *= d_out

        logger.info(f"ChannelChunkConv3d._get_output_numel final {numel=}")
        return numel

    def _get_n_chunks(self, numel: int, n_channels: int):
        """
        チャンク数を計算する
        1チャンクあたりの要素数がCONV3D_NUMEL_LIMIT以下になるようにチャンク数を決定する

        Args:
            numel (int): 演算上の要素数
            n_channels (int): チャンネル数
        Returns:
            int: チャンク数
        """
        logger.info(f"ChannelChunkConv3d._get_n_chunks {numel=} {n_channels=}")

        # 演算上の要素数 / 制限 = チャンク数
        n_chunks = math.ceil(numel / ChannelChunkConv3d.CONV3D_NUMEL_LIMIT)

        # チャンク数がチャンネル数で割り切れるように調整
        n_chunks = ceil_to_divisible(n_chunks, n_channels)

        logger.debug(f"ChannelChunkConv3d._get_n_chunks {n_chunks=}")

        return n_chunks

    def forward(self, input: Tensor) -> Tensor:
        """
        入力テンソルに対してチャンク化されたConv3dを実行する
        """
        logger.info(f"ChannelChunkConv3d.forward {input.shape=}")

        # 要素数が制限以下の場合、通常のConv3dを実行
        if input.numel() // input.size(0) < ChannelChunkConv3d.CONV3D_NUMEL_LIMIT:
            return super().forward(input)

        # 入力チャンネルの分割数を計算
        n_in_chunks = self._get_n_chunks(input.numel(), self.in_channels)

        # 出力チャンネルの分割数を計算
        n_out_chunks = self._get_n_chunks(self._get_output_numel(input.shape), self.out_channels)

        # 入力と出力のチャンク数が1の場合、通常のConv3dを実行
        if n_in_chunks == 1 and n_out_chunks == 1:
            return super().forward(input)

        outputs = []

        # 入力をチャンクに分割
        input_shards = input.chunk(n_in_chunks, dim=1)

        #  重みとバイアスをチャンクに分割してConv3dを実行
        for weight, bias in zip(self.weight.chunk(n_out_chunks), self.bias.chunk(n_out_chunks)):

            # 重みを更にチャンクに分割
            weight_shards = weight.chunk(n_in_chunks, dim=1)

            o = None

            # 入力のシャードと重みのシャードを組み合わせてConv3dを実行
            for x, w in zip(input_shards, weight_shards):

                # 初回
                if o is None:
                    # バイアスあり
                    o = F.conv3d(x, w, bias, self.stride, self.padding, self.dilation, self.groups)
                # 2回目以降
                else:
                    # バイアスなし
                    o += F.conv3d(x, w, None, self.stride, self.padding, self.dilation, self.groups)

            outputs.append(o)

        # 出力を結合して返す
        return torch.cat(outputs, dim=1)

In [ ]:
class ConvLayer(nn.Module):
    """
    2Dまたは3D畳み込み層、正規化、活性化関数を組み合わせた汎用モジュール
    build_downsample_block, build_encoder_project_in_block, GLUMBConv
    InterpolateConvUpSamplerLayer, LiteMLA, ResBlockで使用
    """

    def __init__(self, in_channels: int, out_channels: int, kernel_size=3, stride=1, dilation=1, groups=1, use_bias=False, dropout=0, norm="bn2d", act_func="relu", is_video=False, pad_mode_3d="constant"):
        """
        Args:
            in_channels (int): 入力チャネル数
            out_channels (int): 出力チャネル数
            kernel_size (int, optional): 畳み込みカーネルのサイズ
            stride (int, optional): 畳み込みのストライド
            dilation (int, optional): 畳み込みのダイレーション
            groups (int, optional): 畳み込みのグループ数
            use_bias (bool, optional): バイアスを使用するかどうか
            dropout (float, optional): ドロップアウト率
            norm (str, optional): 正規化層の種類
            act_func (str, optional): 活性化関数の種類
            is_video (bool, optional): 3D畳み込みを使用するかどうか
            pad_mode_3d (str, optional): 3D畳み込みのパディングモード
        """
        logger.info(f"ConvLayer.__init__ {in_channels=} {out_channels=} {kernel_size=} {stride=} {dilation=} {groups=} {use_bias=} {dropout=} {norm=} {act_func=} {is_video=} {pad_mode_3d=}")

        # 1) 初期化

        super().__init__()

        self.is_video = is_video # True

        if self.is_video:
            assert dilation == 1, "only support dilation=1 for 3d conv"
            assert kernel_size % 2 == 1, "only support odd kernel size for 3d conv"

            # 1-1) パディングの設定

            # HanyuanのCausalConv3dに合わせた設定
            # constant
            self.pad_mode_3d = pad_mode_3d

            # kernel_size=1のとき、0,0,0,0,0,0
            # kernel_size=3のとき、1,1,1,1,1,1
            padding = (
                kernel_size // 2,
                kernel_size // 2,
                kernel_size // 2,
                kernel_size // 2,
                kernel_size // 2,
                kernel_size // 2,
            )

            self.padding = padding

            # 1-2) ドロップアウトの設定

            # None
            self.dropout = nn.Dropout3d(dropout, inplace=False) if dropout > 0 else None

            assert isinstance(stride, (int, tuple)), "stride must be an integer or 3-tuple for 3d conv"


            # 1-3) 3D畳み込み層の設定

            # padding is handled by F.pad() in forward()
            self.conv = ChannelChunkConv3d(
                in_channels, # 3など
                out_channels, # 128など
                kernel_size=(kernel_size, kernel_size, kernel_size), # (3,3,3)など
                stride=(stride, stride, stride) if isinstance(stride, int) else stride, # (1,1,1)など
                groups=groups, # 1
                bias=use_bias, # Trueなど
            )

        # False
        else:
            padding = get_same_padding(kernel_size)
            padding *= dilation
            self.dropout = nn.Dropout2d(dropout, inplace=False) if dropout > 0 else None
            self.conv = nn.Conv2d(
                in_channels,
                out_channels,
                kernel_size=(kernel_size, kernel_size),
                stride=(stride, stride),
                padding=padding,
                dilation=(dilation, dilation),
                groups=groups,
                bias=use_bias,
            )

        # 2) 正規化層の設定

        # rms3d or None
        self.norm = build_norm(norm, num_features=out_channels)

        # 3) 活性化関数の設定

        # silu or None
        self.act = build_act(act_func)

        # 4) パディング関数の設定

        self.pad = F.pad

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Args:
            x (torch.Tensor): 入力テンソル
        Returns:
            torch.Tensor: 出力テンソル
        """
        logger.info(f"ConvLayer.forward {x.shape=}")

        # 1) ドロップアウトの適用

        # False
        if self.dropout is not None:
            x = self.dropout(x)

        # 2) 3D畳み込みのカスタムパディングの適用 

        # True
        if self.is_video:
            # constantの場合、デフォルトで0でパディング
            x = self.pad(x, self.padding, mode=self.pad_mode_3d)
            logger.debug(f"ConvLayer.forward after pad {x.shape=}")

        # 3) 畳み込みの適用

        x = self.conv(x)
        logger.debug(f"ConvLayer.forward after conv {x.shape=}")

        # 4) 正規化の適用

        if self.norm:
            # rms3d
            x = self.norm(x)

        # 5) 活性化関数の適用

        if self.act:
            # silu
            x = self.act(x)

        return x

In [ ]:
class GLUMBConv(nn.Module):
    """
    GLU(Gated Liner Unit)を組み込んだInverted Bottleneck Convolutionモジュール
    EfficientViTBlockのローカルモジュールとして使用し、局所的な特徴を抽出能力を高める
    合計18回使用

    1. 1x1畳み込みでチャネル数を拡張
    2. Depthwise畳み込みで空間的特徴を抽出
    3. GLU活性化関数で情報をゲート
    4. 1x1畳み込みでチャネル数を圧縮し、元のチャネル数に戻す
    """

    def __init__(self, in_channels: int, out_channels: int, kernel_size=3, stride=1, mid_channels=None, expand_ratio=6, use_bias=False, norm=(None, None, "ln2d"), act_func=("silu", "silu", None), is_video=False):
        """
        Args:
            in_channels (int): 入力チャネル数
            out_channels (int): 出力チャネル数
            kernel_size (int, optional): 畳み込みカーネルのサイズ
            stride (int, optional): 畳み込みのストライド
            mid_channels (int, optional): 中間チャネル数
            expand_ratio (float, optional): 拡張比率
            use_bias (bool or tuple, optional): バイアスを使用するかどうか
            norm (str or tuple, optional): 正規化層の種類
            act_func (str or tuple, optional): 活性化関数の種類
            is_video (bool, optional): 3D畳み込みを使用するかどうか
        """
        logger.info(f"GLUMBConv.__init__ {in_channels=} {out_channels=} {kernel_size=} {stride=} {mid_channels=} {expand_ratio=} {use_bias=} {norm=} {act_func=} {is_video=}")

        # 1) 初期化

        super().__init__()

        # (True, True, False)
        use_bias = val2tuple(use_bias, 3)

        # (None, None, "rms3d")
        norm = val2tuple(norm, 3)

        # ("silu", "silu", None)
        act_func = val2tuple(act_func, 3)

        # 512 * 4 = 2048など
        mid_channels = round(in_channels * expand_ratio) if mid_channels is None else mid_channels

        # 2) GLU活性化関数の設定

        self.glu_act = build_act(act_func[1], inplace=False)

        # 3) チャンネル数拡張（inverted）用の1x1畳み込み層の構築

        self.inverted_conv = ConvLayer(
            in_channels, # 512など
            mid_channels * 2, # 2048 * 2 = 4096など
            1,
            use_bias=use_bias[0], # True
            norm=norm[0], # None
            act_func=act_func[0], # "silu"
            is_video=is_video, # True
        )

        # 4) Depthwise畳み込み層の構築

        self.depth_conv = ConvLayer(
            mid_channels * 2, # 2048 * 2 = 4096など
            mid_channels * 2, # 2048 * 2 = 4096など
            kernel_size, # 3
            stride=stride, # 1
            groups=mid_channels * 2, # 2048 * 2 = 4096など
            use_bias=use_bias[1], # True
            norm=norm[1], # None
            act_func=None,
            is_video=is_video, # True
        )

        # 5) チャンネル数圧縮用のPointwise（1x1）畳み込み層の構築

        self.point_conv = ConvLayer(
            mid_channels, # 2048など
            out_channels, # 512など
            1,
            use_bias=use_bias[2], # False
            norm=norm[2], # rms3d
            act_func=act_func[2], # None
            is_video=is_video, # True
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        logger.info(f"GLUMBConv.forward {x.shape=}")

        # 1) チャンネル数拡張（inverted）

        x = self.inverted_conv(x)
        logger.debug(f"GLUMBConv.forward after inverted_conv {x.shape=}")

        # 2) Depthwise畳み込み

        x = self.depth_conv(x)
        logger.debug(f"GLUMBConv.forward after depth_conv {x.shape=}")

        # 3) GLU活性化関数の適用

        # チャネルを2つに分割
        x, gate = torch.chunk(x, 2, dim=1)

        # GLU活性化関数(silu)を適用し、情報の通過量のゲートを生成
        gate = self.glu_act(gate)

        # 要素ごとに乗算してゲートを適用
        x = x * gate

        logger.debug(f"GLUMBConv.forward after GLU {x.shape=}")

        # 4) チャンネル数圧縮

        x = self.point_conv(x)

        logger.debug(f"GLUMBConv.forward after point_conv {x.shape=}")

        return x

### ピクセルシャッフリング

In [ ]:
def pixel_unshuffle_3d(x, downsample_factor):
    """
    3Dピクセルアンシャッフル

    Args:
        x (torch.Tensor): 入力テンソル (B, C, T, H, W)
        downsample_factor (int): ダウンサンプリング因子
    Returns:
        torch.Tensor: 出力テンソル (B, C * r^3, T/r, H/r, W/r)
    """
    logger.info(f"pixel_unshuffle_3d {x.shape=} {downsample_factor=}")

    B, C, T, H, W = x.shape

    r = downsample_factor
    assert T % r == 0, f"时间维度必须是下采样因子的倍数, got shape {x.shape}"
    assert H % r == 0, f"高度维度必须是下采样因子的倍数, got shape {x.shape}"
    assert W % r == 0, f"宽度维度必须是下采样因子的倍数, got shape {x.shape}"

    # 時間, 高さ, 幅の各次元をrで分割
    T_new = T // r
    H_new = H // r
    W_new = W // r

    # チャネル数をr^3倍に増加
    C_new = C * (r * r * r)

    # rで分割
    # (B, C, T, H, W) -> (B, C, T_new, r, H_new, r, W_new, r)
    x = x.view(B, C, T_new, r, H_new, r, W_new, r)

    # (B, C, T_new, r, H_new, r, W_new, r) -> (B, C, r, r, r, T_new, H_new, W_new)
    x = x.permute(0, 1, 3, 5, 7, 2, 4, 6)

    # チャンネル次元を結合
    # (B, C, r, r, r, T_new, H_new, W_new) -> (B, C * r^3, T_new, H_new, W_new)
    y = x.reshape(B, C_new, T_new, H_new, W_new)

    logger.debug(f"pixel_unshuffle_3d output {y.shape=}")
    return y

In [ ]:
class PixelUnshuffleChannelAveragingDownSampleLayer(nn.Module):
    """
    ピクセルアンシャッフルとチャネル平均化によるダウンサンプリング層
    build_downsample_blockで5回使用し、build_encoder_project_out_blockで1回使用
    build_downsample_blockでは、畳み込み層の残差接続のショートカットとして使用
    """

    def __init__(self, in_channels: int, out_channels: int, factor: int, temporal_downsample: bool = False):
        """
        Args:
            in_channels (int): 入力チャネル数
            out_channels (int): 出力チャネル数
            factor (int): ダウンサンプリングファクター
            temporal_downsample (bool, optional):
                5D入力テンソルに対して時間方向のダウンサンプリングを行うかどうか
        """
        logger.info(f"PixelUnshuffleChannelAveragingDownSampleLayer.__init__ {in_channels=} {out_channels=} {factor=} {temporal_downsample=}")

        super().__init__()

        self.in_channels = in_channels # 128など
        self.out_channels = out_channels # 256など
        self.factor = factor # 2など
        self.temporal_downsample = temporal_downsample # Falseなど

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        logger.info(f"PixelUnshuffleChannelAveragingDownSampleLayer.forward {x.shape=}")

        # (1, 128, 4, 256, 256)など

        # False
        if x.dim() == 4:
            assert self.in_channels * self.factor**2 % self.out_channels == 0
            group_size = self.in_channels * self.factor**2 // self.out_channels
            x = F.pixel_unshuffle(x, self.factor)
            B, C, H, W = x.shape
            x = x.view(B, self.out_channels, group_size, H, W)
            x = x.mean(dim=2)

        # True
        elif x.dim() == 5:  # [B, C, T, H, W]

            # 1) ピクセルアンシャッフルの適用

            # 4など
            _, _, T, _, _ = x.shape

            # True or False
            if self.temporal_downsample and T != 1:
                # 3Dピクセルアンシャッフル
                x = pixel_unshuffle_3d(x, self.factor)

                assert self.in_channels * self.factor**3 % self.out_channels == 0

                # チャンネル平均化のためのグループサイズ
                group_size = self.in_channels * self.factor**3 // self.out_channels
            else:
                # (B, C, T, H, W) -> (B, T, C, H, W)
                x = x.permute(0, 2, 1, 3, 4)

                # 2Dピクセルアンシャッフル
                x = F.pixel_unshuffle(x, self.factor)

                # (B, T, C_new, H_new, W_new) -> (B, C_new, T, H_new, W_new)
                x = x.permute(0, 2, 1, 3, 4)

                assert self.in_channels * self.factor**2 % self.out_channels == 0

                # チャンネル平均化のためのグループサイズ
                group_size = self.in_channels * self.factor**2 // self.out_channels

            B, C, T, H, W = x.shape
            logger.debug(f"After pixel unshuffle: {x.shape=}")

            # (B, C, T, H, W) -> (B, C_out, group_size, T, H, W)
            x = x.view(B, self.out_channels, group_size, T, H, W)

            # 2) チャネル平均化

            # (B, C_out, group_size, T, H, W) -> (B, C_out, T, H, W)
            x = x.mean(dim=2)
            logger.debug(f"After channel averaging: {x.shape=}")

        # False
        else:
            raise ValueError(f"Unsupported input dimension: {x.dim()}")
        return x

    def __repr__(self):
        return f"PixelUnshuffleChannelAveragingDownSampleLayer(in_channels={self.in_channels}, out_channels={self.out_channels}, factor={self.factor}), temporal_downsample={self.temporal_downsample}"


In [ ]:
def pixel_shuffle_3d(x, upscale_factor):
    """
    3Dピクセルシャッフル

    Args:
        x (torch.Tensor): 入力テンソル (B, C, T, H, W)
        upscale_factor (int): アップサンプリング因子
    Returns:
        torch.Tensor: 出力テンソル (B, C / r^3, T*r, H*r, W*r)
    """
    logger.info(f"pixel_shuffle_3d {x.shape=} {upscale_factor=}")

    B, C, T, H, W = x.shape
    r = upscale_factor
    assert C % (r * r * r) == 0, "通道数必须是上采样因子的立方倍数"

    # チャネル数をr^3で減少
    C_new = C // (r * r * r)

    # チャネル次元を分割
    # (B, C, T, H, W) -> (B, C_new, r, r, r, T, H, W)
    x = x.view(B, C_new, r, r, r, T, H, W)

    # (B, C_new, r, r, r, T, H, W) -> (B, C_new, T, r, H, r, W, r)
    x = x.permute(0, 1, 5, 2, 6, 3, 7, 4)

    # rで結合
    # (B, C_new, T, r, H, r, W, r) -> (B, C_new, T * r, H * r, W * r)
    y = x.reshape(B, C_new, T * r, H * r, W * r)
    return y

In [ ]:
class ChannelDuplicatingPixelShuffleUpSampleLayer(nn.Module):
    """
    チャンネル複製とピクセルシャッフルによるアップサンプリング層
    build_upsample_block, build_decoder_project_in_blockで使用
    """

    def __init__(self, in_channels: int, out_channels: int, factor: int, temporal_upsample: bool = False):
        """
        Args:
            in_channels (int): 入力チャネル数
            out_channels (int): 出力チャネル数
            factor (int): アップサンプリングファクター
            temporal_upsample (bool, optional):
                5D入力テンソルに対して時間方向のアップサンプリングを行うかどうか
        """
        logger.info(f"ChannelDuplicatingPixelShuffleUpSampleLayer.__init__ {in_channels=} {out_channels=} {factor=} {temporal_upsample=}")
        super().__init__()

        # 入力チャネル数
        # 例: 128
        self.in_channels = in_channels

        # 出力チャネル数
        # 例: 1024
        self.out_channels = out_channels

        # アップサンプリングファクター
        # 例: 1
        self.factor = factor

        assert out_channels * factor**2 % in_channels == 0

        # 時間方向のアップサンプリングを行うかどうか
        # 例: False
        self.temporal_upsample = temporal_upsample

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        logger.info(f"ChannelDuplicatingPixelShuffleUpSampleLayer.forward {x.shape=}")

        # 例: (1, 128, 1, 8, 8)

        # 1) 入力を複製

        # True
        if x.dim() == 5:
            B, C, T, H, W = x.shape
            assert C == self.in_channels

        # False
        if self.temporal_upsample and T != 1:  # video input
            repeats = self.out_channels * self.factor**3 // self.in_channels
            logger.debug(f"temporal_upsample repeats (T!=1): {repeats}")
        else:
            repeats = self.out_channels * self.factor**2 // self.in_channels
            logger.debug(f"temporal_upsample repeats (T==1): {repeats}")

        # 入力をチャネル方向に繰り返し複製
        x = x.repeat_interleave(repeats, dim=1)
        logger.debug(f"after repeat_interleave: {x.shape}")

        # 2) ピクセルシャッフルでアップサンプリング

        # False
        if x.dim() == 4:  # original image-only training
            x = F.pixel_shuffle(x, self.factor)

        # True
        elif x.dim() == 5:  # [B, C, T, H, W]

            # True or False
            if self.temporal_upsample and T != 1:

                # 3Dピクセルシャッフルを適用
                x = pixel_shuffle_3d(
                    x,
                    self.factor # 1 or 2
                )

            else:
                # (B, C, T, H, W) -> (B, T, C, H, W)
                x = x.permute(0, 2, 1, 3, 4)

                # 2Dピクセルシャッフルを適用
                x = F.pixel_shuffle(
                    x,
                    self.factor # 1 or 2
                )
                logger.debug(f"after pixel_shuffle: {x.shape}")

                # (B, T, C_new, H_new, W_new) -> (B, C_new, T, H_new, W_new)
                x = x.permute(0, 2, 1, 3, 4)
        return x

    def __repr__(self):
        return f"ChannelDuplicatingPixelShuffleUpSampleLayer(in_channels={self.in_channels}, out_channels={self.out_channels}, factor={self.factor}, temporal_upsample={self.temporal_upsample})"

### InterpolateConvUpSamplerLayer

In [ ]:
VERBOSE = False

def chunked_interpolate(x, scale_factor, mode="nearest"):
    """
    チャンネルの次元に沿ってチャンク化して大きなテンソルを補間する
    現在は「nearest」補間モードのみサポート

    https://discuss.pytorch.org/t/error-using-f-interpolate-for-large-3d-input/207859

    Args:
        x (torch.Tensor): 入力テンソル (B, C, D, H, W)
        scale_factor: スケーリングファクターのタプル (d, h, w)
    Returns:
        torch.Tensor: 補間されたテンソル
    """
    logger.info(f"chunked_interpolate {x.shape=} {scale_factor=} {mode=}")

    assert (
        mode == "nearest"
    ), "Only the nearest mode is supported"  # actually other modes are theoretically supported but not tested
    if len(x.shape) != 5:
        raise ValueError("Expected 5D input tensor (B, C, D, H, W)")

    # Calculate max chunk size to avoid int32 overflow. num_elements < max_int32
    # Max int32 is 2^31 - 1
    max_elements_per_chunk = 2**31 - 1

    # Calculate output spatial dimensions
    out_d = math.ceil(x.shape[2] * scale_factor[0])
    out_h = math.ceil(x.shape[3] * scale_factor[1])
    out_w = math.ceil(x.shape[4] * scale_factor[2])

    # Calculate max channels per chunk to stay under limit
    elements_per_channel = out_d * out_h * out_w
    max_channels = max_elements_per_chunk // (x.shape[0] * elements_per_channel)

    # Use smaller of max channels or input channels
    chunk_size = min(max_channels, x.shape[1])

    # Ensure at least 1 channel per chunk
    chunk_size = max(1, chunk_size)
    if VERBOSE:
        print(f"Input channels: {x.shape[1]}")
        print(f"Chunk size: {chunk_size}")
        print(f"max_channels: {max_channels}")
        print(f"num_chunks: {math.ceil(x.shape[1] / chunk_size)}")

    chunks = []
    for i in range(0, x.shape[1], chunk_size):
        start_idx = i
        end_idx = min(i + chunk_size, x.shape[1])

        chunk = x[:, start_idx:end_idx, :, :, :]

        interpolated_chunk = F.interpolate(chunk, scale_factor=scale_factor, mode="nearest")

        chunks.append(interpolated_chunk)

    if not chunks:
        raise ValueError(f"No chunks were generated. Input shape: {x.shape}")

    # Concatenate chunks along channel dimension
    return torch.cat(chunks, dim=1)

class InterpolateConvUpSampleLayer(nn.Module):
    """
    補間と畳み込みを組み合わせたアップサンプリング層
    build_upsample_blockで使用
    """

    def __init__(self, in_channels: int, out_channels: int, kernel_size: int, factor: int, mode: str = "nearest", is_video: bool = False, temporal_upsample: bool = False) -> None:
        """
        Args:
            in_channels (int): 入力チャネル数
            out_channels (int): 出力チャネル数
            kernel_size (int): 畳み込みカーネルのサイズ
            factor (int): アップサンプリングファクター
            mode (str, optional): 補間モード
            is_video (bool, optional): 3D畳み込みを使用するかどうか
            temporal_upsample (bool, optional):
                5D入力テンソルに対して時間方向のアップサンプリングを行うかどうか
        """
        super().__init__()
        self.factor = factor
        self.mode = mode
        self.temporal_upsample = temporal_upsample
        self.conv = ConvLayer(
            in_channels=in_channels,
            out_channels=out_channels,
            kernel_size=kernel_size,
            use_bias=True,
            norm=None,
            act_func=None,
            is_video=is_video,
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        logger.info(f"InterpolateConvUpSampleLayer.forward {x.shape=}")
        if x.dim() == 4:
            x = F.interpolate(x, scale_factor=self.factor, mode=self.mode)
        elif x.dim() == 5:
            # [B, C, T, H, W] -> [B, C, T*factor, H*factor, W*factor]
            if self.temporal_upsample and x.size(2) != 1:  # temporal upsample for video input
                x = chunked_interpolate(x, scale_factor=[self.factor, self.factor, self.factor], mode=self.mode)
            else:
                x = chunked_interpolate(x, scale_factor=[1, self.factor, self.factor], mode=self.mode)
        x = self.conv(x)
        return x

    def __repr__(self):
        return f"InterpolateConvUpSampleLayer(factor={self.factor}, mode={self.mode}, temporal_upsample={self.temporal_upsample})"

### アテンション機構

In [ ]:
class LiteMLA(nn.Module):
    """
    軽量なマルチスケール線形アテンション機構（Lightweight Multi-scale Linear Attention）
    EfficientViTBlockで合計18回使用
    SoftmaxではなくReLUを使用し、行列積の計算順序を変えることで計算コストをO(N^2)からO(N)に削減
    スケール時に畳み込みを使用することで、局所的な情報を捉える
    """

    def __init__(self, in_channels: int, out_channels: int, heads: Optional[int] = None, heads_ratio: float = 1.0, dim=8, use_bias=False, norm=(None, "bn2d"), act_func=(None, None), kernel_func="relu", scales: tuple[int, ...] = (5,), eps=1.0e-15, is_video=False):
        """
        Args:
            in_channels (int): 入力チャネル数
            out_channels (int): 出力チャネル数
            heads (Optional[int], optional): アテンションヘッド数
            heads_ratio (float, optional): ヘッド数の比率
            dim (int, optional): 各ヘッドの次元数
            use_bias (bool or tuple, optional): バイアスを使用するかどうか
            norm (str or tuple, optional): 正規化層の種類
            act_func (str or tuple, optional): 活性化関数の種類
            kernel_func (str, optional): カーネル関数の種類
            scales (tuple[int, ...], optional): マルチスケール畳み込みのスケール
            eps (float, optional): 数値安定性のための小さな値
            is_video (bool, optional): 3D畳み込みを使用するかどうか
        """
        logger.info(f"LiteMLA.__init__ {in_channels=} {out_channels=} {heads=} {heads_ratio=} {dim=} {use_bias=} {norm=} {act_func=} {kernel_func=} {scales=} {eps=} {is_video=}")

        # 1) 初期化

        super().__init__()

        self.eps = eps # 1e-15

        # 512 // 32 * 1.0 = 32など
        heads = int(in_channels // dim * heads_ratio) if heads is None else heads

        # 32 * 32 = 1024など
        total_dim = heads * dim

        # (False, False)
        use_bias = val2tuple(use_bias, 2)

        # (rms3d, rms3d)
        norm = val2tuple(norm, 2)

        # (None, None)
        act_func = val2tuple(act_func, 2)

        # 32
        self.dim = dim

        # 2) Q, K, Vを生成する畳み込み層を構築

        self.qkv = ConvLayer(
            in_channels,
            3 * total_dim, # 1024 * 3 = 3072など
            1,
            use_bias=use_bias[0], # False
            norm=norm[0], # rms3d
            act_func=act_func[0], # None
            is_video=is_video, # True
        )

        # 3) マルチスケール畳み込み層を構築

        # ChannelChunkConv3d
        conv_class = nn.Conv2d if not is_video else ChannelChunkConv3d

        # スケールごとに畳み込み層を構築（合計1つ）
        self.aggreg = nn.ModuleList(
            [
                nn.Sequential(
                    conv_class(
                        3 * total_dim, # 1024 * 3 = 3072など
                        3 * total_dim, # 1024 * 3 = 3072など
                        scale, # 5 
                        padding=get_same_padding(scale), # 2
                        groups=3 * total_dim, # 1024 * 3 = 3072など
                        bias=use_bias[0], # False
                    ),
                    conv_class(
                        3 * total_dim, # 1024 * 3 = 3072など
                        3 * total_dim, # 1024 * 3 = 3072など
                        1,
                        groups=3 * heads, # 32 * 3 = 96など
                        bias=use_bias[0] # False
                    ),
                )
                for scale in scales
            ]
        )

        # 4) カーネル関数の構築

        # relu
        self.kernel_func = build_act(kernel_func, inplace=False)

        # 5) 出力投影層の構築

        self.proj = ConvLayer(
            total_dim * (1 + len(scales)), # 1024 * (1 + 1) = 2048など
            out_channels, # 1024など
            1,
            use_bias=use_bias[1], # False
            norm=norm[1], # rms3d
            act_func=act_func[1], # None
            is_video=is_video, # True
        )

    @torch.autocast(device_type="cuda", enabled=False)
    def relu_linear_att(self, qkv: torch.Tensor) -> torch.Tensor:
        """
        ReLUカーネルを用いた線形アテンション
        """
        logger.info(f"LiteMLA.relu_linear_att {qkv.shape=}")

        # 1) Q, K, Vの分割と整形

        if qkv.ndim == 5:
            B, _, T, H, W = list(qkv.size())
            is_video = True
        else:
            B, _, H, W = list(qkv.size())
            is_video = False

        if qkv.dtype == torch.float16:
            qkv = qkv.float()

        if qkv.ndim == 4:
            qkv = torch.reshape(
                qkv,
                (
                    B,
                    -1,
                    3 * self.dim,
                    H * W,
                ),
            )
        elif qkv.ndim == 5:
            qkv = torch.reshape(
                qkv,
                (
                    B,
                    -1,
                    3 * self.dim,
                    H * W * T,
                ),
            )

        logger.debug(f"LiteMLA.relu_linear_att after reshape {qkv.shape=}")

        q, k, v = (
            qkv[:, :, 0 : self.dim],
            qkv[:, :, self.dim : 2 * self.dim],
            qkv[:, :, 2 * self.dim :],
        )

        logger.debug(f"LiteMLA.relu_linear_att after split {q.shape=} {k.shape=} {v.shape=}")

        # 2) 軽量線形アテンションの計算

        q = self.kernel_func(q)
        k = self.kernel_func(k)
        logger.debug(f"{q.shape=} {k.shape=} after kernel_func")

        trans_k = k.transpose(-1, -2)

        v = F.pad(v, (0, 0, 0, 1), mode="constant", value=1)
        logger.debug(f"LiteMLA.relu_linear_att after v pad {v.shape=}")

        # キーとバリューの行列積を先に計算
        # トークン数に依存しないため軽量
        vk = torch.matmul(v, trans_k)
        logger.debug(f"LiteMLA.relu_linear_att after vk matmul {vk.shape=}")

        # クエリーと(vk)の行列積を計算
        out = torch.matmul(vk, q)
        logger.debug(f"LiteMLA.relu_linear_att after out matmul {out.shape=}")

        # 3) 正規化と整形

        if out.dtype == torch.bfloat16:
            out = out.float()

        out = out[:, :, :-1] / (out[:, :, -1:] + self.eps)
        logger.debug(f"LiteMLA.relu_linear_att after normalization {out.shape=}")

        # False
        if not is_video:
            out = torch.reshape(out, (B, -1, H, W))
        # True
        else:
            out = torch.reshape(out, (B, -1, T, H, W))

        logger.debug(f"LiteMLA.relu_linear_att after reshape {out.shape=}")

        return out

    @torch.autocast(device_type="cuda", enabled=False)
    def relu_quadratic_att(self, qkv: torch.Tensor) -> torch.Tensor:
        """
        Quadratic Attention with ReLU kernel
        今回は使用しない
        """
        B, _, H, W = list(qkv.size())

        qkv = torch.reshape(
            qkv,
            (
                B,
                -1,
                3 * self.dim,
                H * W,
            ),
        )
        q, k, v = (
            qkv[:, :, 0 : self.dim],
            qkv[:, :, self.dim : 2 * self.dim],
            qkv[:, :, 2 * self.dim :],
        )

        q = self.kernel_func(q)
        k = self.kernel_func(k)

        att_map = torch.matmul(k.transpose(-1, -2), q)  # b h n n
        original_dtype = att_map.dtype
        if original_dtype in [torch.float16, torch.bfloat16]:
            att_map = att_map.float()
        att_map = att_map / (torch.sum(att_map, dim=2, keepdim=True) + self.eps)  # b h n n
        att_map = att_map.to(original_dtype)
        out = torch.matmul(v, att_map)  # b h d n

        out = torch.reshape(out, (B, -1, H, W))
        return out

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Args:
            x (torch.Tensor): 入力テンソル
        Returns:
            torch.Tensor: 出力テンソル
        """

        logger.info(f"LiteMLA.forward {x.shape=}")

        # 1) マルチスケールQKVの生成

        qkv = self.qkv(x)

        multi_scale_qkv = [qkv]

        for op in self.aggreg:
            multi_scale_qkv.append(op(qkv))

        qkv = torch.cat(multi_scale_qkv, dim=1)

        logger.debug(f"LiteMLA.forward after multi-scale conv {qkv.shape=}")

        # 2) ReLU線形アテンションの適用

        out = self.relu_linear_att(qkv).to(qkv.dtype)
        logger.debug(f"LiteMLA.forward after relu_linear_att {out.shape=}")

        # 3) 出力投影の適用

        out = self.proj(out)
        logger.debug(f"LiteMLA.forward after proj {out.shape=}")

        return out

### ResidualBlock

In [ ]:
class ResidualBlock(nn.Module):
    """
    ブロックに残差接続を追加するモジュール
    build_downsample_block, build_encoder_project_out_block, build_block,
    build_decoder_project_in_block, EfficientViTBlockで使用
    """

    def __init__(
        self,
        main: Optional[nn.Module],
        shortcut: Optional[nn.Module],
        post_act=None,
        pre_norm: Optional[nn.Module] = None,
    ):
        logger.info(f"ResidualBlock.__init__ {main=} {shortcut=} {post_act=} {pre_norm=}")

        super().__init__()

        self.pre_norm = pre_norm
        self.main = main
        self.shortcut = shortcut
        self.post_act = build_act(post_act)

    def forward_main(self, x: torch.Tensor) -> torch.Tensor:
        if self.pre_norm is None:
            return self.main(x)
        else:
            return self.main(self.pre_norm(x))

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Args:
            x (torch.Tensor): 入力テンソル
        Returns:
            torch.Tensor: 出力テンソル
        """

        # メインブランチがNoneの場合
        if self.main is None:
            res = x

        # ショートカットブランチがNoneの場合
        elif self.shortcut is None:
            res = self.forward_main(x)

        # 両方のブランチが存在する場合
        else:
            # メインブランチとショートカットブランチの出力を加算（残差接続）
            res = self.forward_main(x) + self.shortcut(x)

            # 活性化関数が存在する場合
            if self.post_act:
                # 活性化関数を適用
                res = self.post_act(res)
        return res

### EfficientViT

In [ ]:
class EfficientViTBlock(nn.Module):
    """
    EfficientViTブロック
    build_blockで18回使用
    """
    def __init__(self, in_channels: int, heads_ratio: float = 1.0, dim=32, expand_ratio: float = 4, scales: tuple[int, ...] = (5,), norm: str = "bn2d", act_func: str = "hswish", context_module: str = "LiteMLA", local_module: str = "MBConv", is_video: bool = False):
        """
        Args:
            in_channels (int): 入力チャネル数
            heads_ratio (float, optional): ヘッド数の比率
            dim (int, optional): 各ヘッドの次元数
            expand_ratio (float, optional): 拡張比率
            scales (tuple[int, ...], optional): マルチスケール畳み込みのスケール
            norm (str, optional): 正規化層の種類
            act_func (str, optional): 活性化関数の種類
            context_module (str, optional): コンテキストモジュールの種類
            local_module (str, optional): ローカルモジュールの種類
            is_video (bool, optional): 3D畳み込みを使用するかどうか
        """
        logger.info(f"EfficientViTBlock.__init__ {in_channels=} {heads_ratio=} {dim=} {expand_ratio=} {scales=} {norm=} {act_func=} {context_module=} {local_module=} {is_video=}")

        super().__init__()

        # 1) コンテキストモジュールを構築

        # True
        if context_module == "LiteMLA":
            self.context_module = ResidualBlock(

                # 残差接続つきのマルチスケール線形アテンションを構築
                LiteMLA(
                    in_channels=in_channels, # 512など
                    out_channels=in_channels, # 512など
                    heads_ratio=heads_ratio, # 1.0
                    dim=dim, # 32
                    norm=(None, norm), # (None, rms3d)
                    scales=scales, # (5,)
                    is_video=is_video, # True
                ),
                IdentityLayer(),
            )

        # False
        else:
            raise ValueError(f"context_module {context_module} is not supported")

        # False
        if local_module == "MBConv":
            self.local_module = ResidualBlock(
                MBConv(
                    in_channels=in_channels,
                    out_channels=in_channels,
                    expand_ratio=expand_ratio,
                    use_bias=(True, True, False),
                    norm=(None, None, norm),
                    act_func=(act_func, act_func, None),
                    is_video=is_video,
                ),
                IdentityLayer(),
            )

        # 2) ローカルモジュールを構築

        # True
        elif local_module == "GLUMBConv":

             # GLUを組み込んだボトルネック拡張畳み込みを構築し、残差接続を追加
            self.local_module = ResidualBlock(
                GLUMBConv(
                    in_channels=in_channels, # 512など
                    out_channels=in_channels, # 512など
                    expand_ratio=expand_ratio, # 4
                    use_bias=(True, True, False),
                    norm=(None, None, norm), # (None, None, rms3d)
                    act_func=(act_func, act_func, None), # (silu, silu, None)
                    is_video=is_video, # True
                ),
                IdentityLayer(),
            )

        # False
        else:
            raise NotImplementedError(f"local_module {local_module} is not supported")

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        logger.info(f"EfficientViTBlock.forward {x.shape=}")

        # コンテキストモジュールの適用
        x = self.context_module(x)
        logger.debug(f"After context_module: {x.shape=}")

        # ローカルモジュールの適用
        x = self.local_module(x)
        logger.debug(f"After local_module: {x.shape=}")

        return x

### ResBlock

In [ ]:
class ResBlock(nn.Module):
    """
    2つの畳込み層からなる残差ブロックのメインブランチ
    ショートカットパスは含まない
    build_blockで使用
    """

    def __init__(
        self,
        in_channels: int,
        out_channels: int,
        kernel_size=3,
        stride=1,
        mid_channels=None,
        expand_ratio=1,
        use_bias=False,
        norm=("bn2d", "bn2d"),
        act_func=("relu6", None),
        is_video=False,
    ):
        """
        Args:
            in_channels (int): 入力チャネル数
            out_channels (int): 出力チャネル数
            kernel_size (int, optional): 畳み込みカーネルのサイズ
            stride (int, optional): 畳み込みのストライド
            mid_channels (int, optional): 中間チャネル数
            expand_ratio (float, optional): 拡張比率
            use_bias (bool or tuple, optional): バイアスを使用するかどうか
            norm (str or tuple, optional): 正規化層の種類
            act_func (str or tuple, optional): 活性化関数の種類
            is_video (bool, optional): 3D畳み込みを使用するかどうか
        """
        logger.info(f"ResBlock.__init__ {in_channels=} {out_channels=} {kernel_size=} {stride=} {mid_channels=} {expand_ratio=} {use_bias=} {norm=} {act_func=} {is_video=}")

        # 1) 初期化

        super().__init__()

        use_bias = val2tuple(use_bias, 2) # (True, False)

        norm = val2tuple(norm, 2) # (None, "rms3d")

        act_func = val2tuple(act_func, 2) # ("silu", None)

        # expand_ratioは常に1なので、入力のチャネル数と中間チャネル数は同じ
        mid_channels = round(in_channels * expand_ratio) if mid_channels is None else mid_channels

        # 2) 1つ目の畳み込み層

        self.conv1 = ConvLayer(
            in_channels, # 128など
            mid_channels, # 128など
            kernel_size, # 3
            stride, # 1
            use_bias=use_bias[0], # True
            norm=norm[0], # None
            act_func=act_func[0], # silu
            is_video=is_video, # True
        )

        # 3) 2つ目の畳み込み層

        self.conv2 = ConvLayer(
            mid_channels, # 128など
            out_channels, # 128など
            kernel_size, # 3
            1,
            use_bias=use_bias[1], # False
            norm=norm[1], # rms3d
            act_func=act_func[1], # None
            is_video=is_video, # True
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        logger.info(f"ResBlock.forward {x.shape=}")

        # 1) 1つ目の畳み込み層を通す
        x = self.conv1(x)
        logger.debug(f"After conv1: {x.shape}")

        # 2) 2つ目の畳み込み層を通す

        x = self.conv2(x)
        logger.debug(f"After conv2: {x.shape}")

        return x

### Encoder

In [ ]:
def build_downsample_block(
    block_type: str,
    in_channels: int,
    out_channels: int,
    shortcut: Optional[str],
    is_video: bool,
    temporal_downsample: bool = False,
) -> nn.Module:
    """
    ダウンサンプリングブロックを構築する
    Encoder.__init__で5回使用

    Args:
        block_type (str): ブロックの種類 ("Conv" または "ConvPixelUnshuffle")
        in_channels (int): 入力チャネル数
        out_channels (int): 出力チャネル数
        shortcut (Optional[str]): ショートカットの種類 (None または "averaging")
        is_video (bool): 3D畳み込みを使用するかどうか
        temporal_downsample (bool, optional):
            時間方向のダウンサンプリングを行うかどうか
    Returns:
        nn.Module: ダウンサンプリングブロック
    """
    logger.info(f"build_downsample_block {block_type=} {in_channels=} {out_channels=} {shortcut=} {is_video=} {temporal_downsample=}")

    # 1) 畳込み層を作成

    # True
    if block_type == "Conv":

        # True
        if is_video:
            # False -> False -> False -> True -> True
            if temporal_downsample:
                stride = (2, 2, 2)
            else:
                stride = (1, 2, 2)
        else:
            stride = 2


        block = ConvLayer(
            in_channels=in_channels, # 128 -> 256 -> 256 -> 512 -> 512
            out_channels=out_channels, # 256 -> 512 -> 512 -> 1024 -> 1024
            kernel_size=3,
            stride=stride,
            use_bias=True,
            norm=None,
            act_func=None,
            is_video=is_video, # True
        )

    # False
    elif block_type == "ConvPixelUnshuffle":
        if is_video:
            raise NotImplementedError("ConvPixelUnshuffle downsample is not supported for video")
        block = ConvPixelUnshuffleDownSampleLayer(
            in_channels=in_channels, out_channels=out_channels, kernel_size=3, factor=2
        )

    # False
    else:
        raise ValueError(f"block_type {block_type} is not supported for downsampling")

    # 2) 畳み込み層に残差接続を追加

    # False
    if shortcut is None:
        pass

    # True
    elif shortcut == "averaging":

        # ピクセルアンシャッフルとチャネル平均化によるダウンサンプリング層を作成
        shortcut_block = PixelUnshuffleChannelAveragingDownSampleLayer(
            in_channels=in_channels, # 128 -> 256 -> 256 -> 512 -> 512
            out_channels=out_channels, # 256 -> 512 -> 512 -> 1024 -> 1024
            factor=2,
            temporal_downsample=temporal_downsample # False -> False -> False -> True -> True
        )

        # ダウンサンプリング層を残差接続のショートカットとして組み込む
        block = ResidualBlock(block, shortcut_block)

    # False
    else:
        raise ValueError(f"shortcut {shortcut} is not supported for downsample")

    return block

In [ ]:
def build_encoder_project_in_block(in_channels: int, out_channels: int, factor: int, downsample_block_type: str, is_video: bool):
    """
    エンコーダの入力プロジェクションブロックを構築する
    Encoder.__init__で1回だけ使用される

    Args:
        in_channels (int): 入力チャネル数
        out_channels (int): 出力チャネル数
        factor (int): ダウンサンプリングファクター
        downsample_block_type (str): ダウンサンプリングブロックの種類
        is_video (bool): 3D畳み込みを使用するかどうか
    Returns:
        nn.Module: エンコーダの入力プロジェクションブロック
    """
    logger.info(f"build_encoder_project_in_block {in_channels=} {out_channels=} {factor=} {downsample_block_type=} {is_video=}")

    # 1) 畳み込み層を構築

    # True
    if factor == 1:

        block = ConvLayer(
            in_channels=in_channels, # 3
            out_channels=out_channels, # 128
            kernel_size=3,
            stride=1,
            use_bias=True,
            norm=None,
            act_func=None,
            is_video=is_video, # True
        )

    # False
    elif factor == 2:
        if is_video:
            raise NotImplementedError("Downsample during project_in is not supported for video")
        block = build_downsample_block(
            block_type=downsample_block_type, in_channels=in_channels, out_channels=out_channels, shortcut=None
        )

    # False
    else:
        raise ValueError(f"downsample factor {factor} is not supported for encoder project in")
    return block

In [ ]:
def build_encoder_project_out_block(
    in_channels: int,
    out_channels: int,
    norm: Optional[str],
    act: Optional[str],
    shortcut: Optional[str],
    is_video: bool,
):
    """
    エンコーダの出力プロジェクションブロックを構築する
    Encoderで使用

    Args:
        in_channels (int): 入力チャネル数
        out_channels (int): 出力チャネル数
        norm (Optional[str]): 正規化層の種類
        act (Optional[str]): 活性化関数の種類
        shortcut (Optional[str]): ショートカットの種類 (None または "averaging")
        is_video (bool): 3D畳み込みを使用するかどうか
    Returns:
        nn.Module: エンコーダの出力プロジェクションブロック
    """
    logger.info(f"build_encoder_project_out_block {in_channels=} {out_channels=} {norm=} {act=} {shortcut=} {is_video=}")

    # 1) 畳み込み層のブロックを作成

    block = OpSequential(
        [
            build_norm(norm), # None
            build_act(act), # None
            ConvLayer(
                in_channels=in_channels, # 1024
                out_channels=out_channels, # 128
                kernel_size=3,
                stride=1,
                use_bias=True,
                norm=None,
                act_func=None,
                is_video=is_video, # True
            ),
        ]
    )

    # 2) ブロックに残差接続を追加

    # False
    if shortcut is None:
        pass

    # True
    elif shortcut == "averaging":

        # ピクセルアンシャッフルとチャネル平均化によるダウンサンプリング層を作成
        shortcut_block = PixelUnshuffleChannelAveragingDownSampleLayer(
            in_channels=in_channels, # 1024
            out_channels=out_channels, # 128
            factor=1
        )

        # ダウンサンプリング層を残差接続のショートカットとして組み込む
        block = ResidualBlock(block, shortcut_block)

    # False
    else:
        raise ValueError(f"shortcut {shortcut} is not supported for encoder project out")

    return block

In [ ]:
def build_block(
    block_type: str, in_channels: int, out_channels: int, norm: Optional[str], act: Optional[str], is_video: bool
) -> nn.Module:
    """
    ブロックを構築する
    build_stage_mainで使用
    Args:
        block_type (str): ブロックの種類 ("ResBlock", "EViT_GLU", "EViTS5_GLU")
        in_channels (int): 入力チャネル数
        out_channels (int): 出力チャネル数
        norm (Optional[str]): 正規化層の種類
        act (Optional[str]): 活性化関数の種類
        is_video (bool): 3D畳み込みを使用するかどうか
    Returns:
        nn.Module: ブロック
    """
    logger.info(f"build_block {block_type=} {in_channels=} {out_channels=} {norm=} {act=} {is_video=}")

    # 1) メインブロックを作成

    # True of False
    if block_type == "ResBlock":

        assert in_channels == out_channels

        # 1-1) ResBlockを作成
        main_block = ResBlock(
            in_channels=in_channels, # 128 or 256 or 512 or 1024
            out_channels=out_channels, # 128 or 256 or 512 or 1024
            kernel_size=3,
            stride=1,
            use_bias=(True, False),
            norm=(None, norm), # rms3d
            act_func=(act, None), # silu
            is_video=is_video, # True
        )

        # 1-2) ResBlockに残差接続を追加
        block = ResidualBlock(main_block, IdentityLayer())

    # False
    elif block_type == "EViT_GLU":
        assert in_channels == out_channels
        block = EfficientViTBlock(
            in_channels, norm=norm, act_func=act, local_module="GLUMBConv", scales=(), is_video=is_video
        )

    # True or False
    elif block_type == "EViTS5_GLU":

        assert in_channels == out_channels

        # 1-1) EfficientViTブロックを作成

        block = EfficientViTBlock(
            in_channels, # 128 or 256 or 512 or 1024
            norm=norm, # rms3d
            act_func=act, # silu
            local_module="GLUMBConv",
            scales=(5,),
            is_video=is_video # True
        )

    # False
    else:
        raise ValueError(f"block_type {block_type} is not supported")

    return block

In [ ]:
def build_stage_main(
    width: int, depth: int, block_type: str | list[str], norm: str, act: str, input_width: int, is_video: bool
) -> list[nn.Module]:
    """
    ステージのメインブロックを構築する
    Encoder, Decoderで合計12回使用

    Args:
        width (int): ブロックのチャネル数
        depth (int): ブロックの深さ
        block_type (str or list[str]): ブロックの種類
        norm (str): 正規化層の種類
        act (str): 活性化関数の種類
        input_width (int): 入力チャネル数
        is_video (bool): 3D畳み込みを使用するかどうか
    Returns:
        list[nn.Module]: ステージのメインブロックのリスト
    """
    logger.info(f"build_stage_main {width=} {depth=} {block_type=} {norm=} {act=} {input_width=} {is_video=}")

    assert isinstance(block_type, str) or (isinstance(block_type, list) and depth == len(block_type))

    # 1) 深さの数だけメインブロックを構築

    stage = []

    # 深さでループ（2 or 3）
    for d in range(depth):

        # ブロックタイプを決定
        # ResBlock or EViTS5_GLU
        current_block_type = block_type[d] if isinstance(block_type, list) else block_type

        # ブロックを構築
        block = build_block(
            block_type=current_block_type, # ResBlock or EViTS5_GLU
            in_channels=width if d > 0 else input_width, # 128 or 256 or 512 or 1024
            out_channels=width, # 128 or 256 or 512 or 1024
            norm=norm, # rms3d
            act=act, # silu
            is_video=is_video, # True
        )

        # ブロックをステージに追加
        stage.append(block)

    return stage

In [ ]:
@dataclass
class EncoderConfig:
    in_channels: int = MISSING
    latent_channels: int = MISSING
    width_list: tuple[int, ...] = (128, 256, 512, 512, 1024, 1024)
    depth_list: tuple[int, ...] = (2, 2, 2, 2, 2, 2)
    block_type: Any = "ResBlock"
    norm: str = "rms2d"
    act: str = "silu"
    downsample_block_type: str = "ConvPixelUnshuffle"
    downsample_match_channel: bool = True
    downsample_shortcut: Optional[str] = "averaging"
    out_norm: Optional[str] = None
    out_act: Optional[str] = None
    out_shortcut: Optional[str] = "averaging"
    double_latent: bool = False
    is_video: bool = False
    temporal_downsample: tuple[bool, ...] = ()

In [ ]:
class Encoder(nn.Module):
    """
    6つのステージを持つDCAEのエンコーダ
    DCAE.__init__で使用
    """

    def __init__(self, cfg: EncoderConfig):
        """
        Args:
            cfg (EncoderConfig): エンコーダの設定
        """
        logger.info(f"Encoder.__init__ {cfg=}")

        # 1) 初期化

        super().__init__()

        self.cfg = cfg

        # 6つのステージ
        # [128, 256, 512, 512, 1024, 1024] -> 6
        num_stages = len(cfg.width_list)

        self.num_stages = num_stages

        assert len(cfg.depth_list) == num_stages
        assert len(cfg.width_list) == num_stages

        assert isinstance(cfg.block_type, str) or (
            isinstance(cfg.block_type, list) and len(cfg.block_type) == num_stages
        )

        # 2) 入力プロジェクションブロックを作成

        self.project_in = build_encoder_project_in_block(
            in_channels=cfg.in_channels, # 128
            out_channels=cfg.width_list[0] if cfg.depth_list[0] > 0 else cfg.width_list[1], # 128
            factor=1 if cfg.depth_list[0] > 0 else 2, # 1
            downsample_block_type=cfg.downsample_block_type, # Conv
            is_video=cfg.is_video, # True
        )

        # 3) 6つのステージを作成

        self.stages: list[OpSequential] = []

        # [128, 256, 512, 512, 1024, 1024]の幅と[2, 2, 2, 3, 3, 3]の深さを持つ6つのステージを構築
        for stage_id, (width, depth) in enumerate(zip(cfg.width_list, cfg.depth_list)):

            # ブロックの種類を決定
            # ['ResBlock', 'ResBlock', 'ResBlock', 'EViTS5_GLU', 'EViTS5_GLU', 'EViTS5_GLU']
            block_type = cfg.block_type[stage_id] \
                if isinstance(cfg.block_type, list) else cfg.block_type

            # ステージのメインブロックを構築
            stage = build_stage_main(
                width=width, # 128 -> 256 -> 512 -> 512 -> 1024 -> 1024
                depth=depth, # 2 -> 2 -> 2 -> 3 -> 3 -> 3
                # ResBlock -> ResBlock -> ResBlock -> EViTS5_GLU -> EViTS5_GLU -> EViTS5_GLU
                block_type=block_type,
                norm=cfg.norm, # rms3d
                act=cfg.act, # silu
                input_width=width, # 128 -> 256 -> 512 -> 512 -> 1024 -> 1024
                is_video=cfg.is_video, # True
            )

            # 最後のステージより前のステージの場合
            if stage_id < num_stages - 1 and depth > 0:

                # ダウンサンプルブロックを構築
                downsample_block = build_downsample_block(
                    block_type=cfg.downsample_block_type, # Conv
                    in_channels=width, # 128 -> 256 -> 512 -> 512 -> 1024
                    # 256 -> 512 -> 512 -> 1024 -> 1024
                    out_channels=cfg.width_list[stage_id + 1] \
                        if cfg.downsample_match_channel else width,
                    shortcut=cfg.downsample_shortcut, # averaging
                    is_video=cfg.is_video, # True
                    # False -> False -> False -> True -> True
                    temporal_downsample=cfg.temporal_downsample[stage_id] \
                        if cfg.temporal_downsample != [] else False,
                )

                # ステージにダウンサンプルブロックを追加
                stage.append(downsample_block)

            # ステージをOpSequentialでラップしてリストに追加
            self.stages.append(OpSequential(stage))

        # ステージのリストをnn.ModuleListに変換
        self.stages = nn.ModuleList(self.stages)

        # 4) 出力プロジェクションブロックを作成

        self.project_out = build_encoder_project_out_block(
            in_channels=cfg.width_list[-1], # 1024
            out_channels=2 * cfg.latent_channels if cfg.double_latent else cfg.latent_channels, # 128
            norm=cfg.out_norm, # None
            act=cfg.out_act, # None
            shortcut=cfg.out_shortcut, # averaging
            is_video=cfg.is_video, # True
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        logger.info(f"Encoder.forward {x.shape=}")

        # 1) 入力プロジェクションブロックを通過
        x = self.project_in(x)
        logger.debug(f"Encoder.forward after project_in {x.shape=}")

        # x = auto_grad_checkpoint(self.project_in, x)

        # 2) 6つのステージを順番に通過

        for i, stage in enumerate(self.stages):
            if len(stage.op_list) == 0:
                continue
            x = auto_grad_checkpoint(stage, x)
            logger.debug(f"Encoder.forward after stage {i} {x.shape=}")

        # x = self.project_out(x)

        # 3) 出力プロジェクションブロックを通過    

        # 自動勾配チェックポイントを使用してメモリ使用量を削減
        x = auto_grad_checkpoint(self.project_out, x)
        logger.debug(f"Encoder.forward after project_out {x.shape=}")

        return x

### Decoder

In [ ]:
def build_decoder_project_in_block(in_channels: int, out_channels: int, shortcut: Optional[str], is_video: bool):
    """
    デコーダの入力プロジェクションブロックを構築する
    Decoder.__init__で使用

    Args:
        in_channels (int): 入力チャネル数
        out_channels (int): 出力チャネル数
        shortcut (Optional[str]): ショートカットの種類 (None または "duplicating")
        is_video (bool): 3D畳み込みを使用するかどうか
    Returns:
        nn.Module: デコーダの入力プロジェクションブロック
    """
    logger.info(f"build_decoder_project_in_block {in_channels=} {out_channels=} {shortcut=} {is_video=}")

    # 1) 畳み込みブロックを作成

    block = ConvLayer(
        in_channels=in_channels, # 128
        out_channels=out_channels, # 1024
        kernel_size=3,
        stride=1,
        use_bias=True,
        norm=None,
        act_func=None,
        is_video=is_video, # True
    )

    # 2) 残差接続用のショートカットブロックを作成し、接続

    # False
    if shortcut is None:
        pass

    # True
    elif shortcut == "duplicating":

        # ピクセルシャッフルでチャネルを複製してアップサンプリングするショートカットブロックを作成
        shortcut_block = ChannelDuplicatingPixelShuffleUpSampleLayer(
            in_channels=in_channels, # 128
            out_channels=out_channels, # 1024
            factor=1
        )

        # 畳み込みブロックとショートカットブロックを残差接続で結合
        block = ResidualBlock(block, shortcut_block)

    # False
    else:
        raise ValueError(f"shortcut {shortcut} is not supported for decoder project in")

    return block

In [ ]:
def build_upsample_block(
    block_type: str,
    in_channels: int,
    out_channels: int,
    shortcut: Optional[str],
    is_video: bool,
    temporal_upsample: bool = False,
) -> nn.Module:
    """
    アップサンプリングブロックを構築する
    build_decoder_project_out_block, Decoder.__init__で合計5回使用

    Args:
        block_type (str): ブロックの種類 ("ConvPixelShuffle" または "InterpolateConv")
        in_channels (int): 入力チャネル数
        out_channels (int): 出力チャネル数
        shortcut (Optional[str]): ショートカットの種類 (None または "duplicating")
        is_video (bool): 3D畳み込みを使用するかどうか
        temporal_upsample (bool, optional):
            時間方向のアップサンプリングを行うかどうか
    Returns:
        nn.Module: アップサンプリングブロック
    """
    logger.info(f"build_upsample_block {block_type=} {in_channels=} {out_channels=} {shortcut=} {is_video=} {temporal_upsample=}")

    # 1) アップサンプリングブロックを作成

    # False
    if block_type == "ConvPixelShuffle":
        if is_video:
            raise NotImplementedError("ConvPixelShuffle upsample is not supported for video")
        block = ConvPixelShuffleUpSampleLayer(
            in_channels=in_channels, out_channels=out_channels, kernel_size=3, factor=2
        )

    # True
    elif block_type == "InterpolateConv":
        block = InterpolateConvUpSampleLayer(
            in_channels=in_channels, # 1024 -> 1024 -> 512 -> 512 -> 256
            out_channels=out_channels, # 1024 -> 512 -> 512 -> 256 -> 128
            kernel_size=3,
            factor=2,
            is_video=is_video, # True
            temporal_upsample=temporal_upsample, # True -> True -> False -> False -> False
        )

    # False
    else:
        raise ValueError(f"block_type {block_type} is not supported for upsampling")

    # 2) 残差接続用のショートカットブロックを作成し、接続

    # False
    if shortcut is None:
        pass

    # True
    elif shortcut == "duplicating":

        # ピクセルシャッフルでチャネルを複製してアップサンプリングするショートカットブロックを作成
        shortcut_block = ChannelDuplicatingPixelShuffleUpSampleLayer(
            in_channels=in_channels, # 1024 -> 1024 -> 512 -> 512 -> 256
            out_channels=out_channels, # 1024 -> 512 -> 512 -> 256 -> 128
            factor=2,
            temporal_upsample=temporal_upsample # True -> True -> False -> False -> False
        )

        # アップサンプリングブロックとショートカットブロックを残差接続で結合
        block = ResidualBlock(block, shortcut_block)

    # False
    else:
        raise ValueError(f"shortcut {shortcut} is not supported for upsample")

    return block

In [ ]:
def build_decoder_project_out_block(
    in_channels: int,
    out_channels: int,
    factor: int,
    upsample_block_type: str,
    norm: Optional[str],
    act: Optional[str],
    is_video: bool,
):
    """
    デコーダの出力プロジェクションブロックを構築する
    Decoder.__init__で使用

    Args:
        in_channels (int): 入力チャネル数
        out_channels (int): 出力チャネル数
        factor (int): アップサンプリングファクター
        upsample_block_type (str): アップサンプリングブロックの種類
        norm (Optional[str]): 正規化層の種類
        act (Optional[str]): 活性化関数の種類
        is_video (bool): 3D畳み込みを使用するかどうか
    Returns:
        nn.Module: デコーダの出力プロジェクションブロック
    """
    logger.info(f"build_decoder_project_out_block {in_channels=} {out_channels=} {factor=} {upsample_block_type=} {norm=} {act=} {is_video=}")

    # 1) レイヤー正規化層と活性化関数を作成し、リストに追加

    layers: list[nn.Module] = [
        build_norm(
            norm, # rms3d
            in_channels # 128
        ),
        build_act(act), # relu
    ]

    # 2) アップサンプリングブロックを作成し、リストに追加

    # True
    if factor == 1:

        # 畳み込みレイヤーを作成し、リストに追加
        layers.append(
            ConvLayer(
                in_channels=in_channels, # 128
                out_channels=out_channels, # 3
                kernel_size=3,
                stride=1,
                use_bias=True,
                norm=None,
                act_func=None,
                is_video=is_video, # True
            )
        )

    # False
    elif factor == 2:
        if is_video:
            raise NotImplementedError("Upsample during project_out is not supported for video")
        layers.append(
            build_upsample_block(
                block_type=upsample_block_type, in_channels=in_channels, out_channels=out_channels, shortcut=None
            )
        )

    # False
    else:
        raise ValueError(f"upsample factor {factor} is not supported for decoder project out")

    # 3) OpSequentialでラップして返す

    return OpSequential(layers)

In [ ]:
@dataclass
class DecoderConfig:
    in_channels: int = MISSING
    latent_channels: int = MISSING
    in_shortcut: Optional[str] = "duplicating"
    width_list: tuple[int, ...] = (128, 256, 512, 512, 1024, 1024)
    depth_list: tuple[int, ...] = (2, 2, 2, 2, 2, 2)
    block_type: Any = "ResBlock"
    norm: Any = "rms2d"
    act: Any = "silu"
    upsample_block_type: str = "ConvPixelShuffle"
    upsample_match_channel: bool = True
    upsample_shortcut: str = "duplicating"
    out_norm: str = "rms2d"
    out_act: str = "relu"
    is_video: bool = False
    temporal_upsample: tuple[bool, ...] = ()

In [ ]:
class Decoder(nn.Module):
    """
    デコーダ
    DCAEで使用
    """

    def __init__(self, cfg: DecoderConfig):
        """
        Args:
            cfg (DecoderConfig): デコーダの設定
        """
        logger.info(f"Decoder.__init__ {cfg=}")

        # 1) 初期化

        super().__init__()

        self.cfg = cfg

        # 6つのステージ
        num_stages = len(cfg.width_list)

        self.num_stages = num_stages

        assert len(cfg.depth_list) == num_stages
        assert len(cfg.width_list) == num_stages
        assert isinstance(cfg.block_type, str) or (
            isinstance(cfg.block_type, list) and len(cfg.block_type) == num_stages
        )
        assert isinstance(cfg.norm, str) or (isinstance(cfg.norm, list) and len(cfg.norm) == num_stages)
        assert isinstance(cfg.act, str) or (isinstance(cfg.act, list) and len(cfg.act) == num_stages)

        # 2) 入力プロジェクションブロックを作成

        self.project_in = build_decoder_project_in_block(
            in_channels=cfg.latent_channels,
            out_channels=cfg.width_list[-1],
            shortcut=cfg.in_shortcut,
            is_video=cfg.is_video,
        )

        # 3) 6つのステージを作成

        self.stages: list[OpSequential] = []

        # [128, 256, 512, 512, 1024, 1024]の幅と[2, 2, 2, 3, 3, 3]の深さを持つ6つのステージを「逆順」に構築
        for stage_id, (width, depth) in reversed(list(enumerate(zip(cfg.width_list, cfg.depth_list)))):
            stage = []

            # 最後のステージより前のステージの場合
            if stage_id < num_stages - 1 and depth > 0:

                # アップサンプルブロックを構築
                upsample_block = build_upsample_block(
                    block_type=cfg.upsample_block_type,
                    in_channels=cfg.width_list[stage_id + 1],
                    out_channels=width if cfg.upsample_match_channel else cfg.width_list[stage_id + 1],
                    shortcut=cfg.upsample_shortcut,
                    is_video=cfg.is_video,
                    temporal_upsample=cfg.temporal_upsample[stage_id] if cfg.temporal_upsample != [] else False,
                )

                # ステージにアップサンプルブロックを追加
                stage.append(upsample_block)

            # ブロックの種類を決定
            # ['ResBlock', 'ResBlock', 'ResBlock', 'EViTS5_GLU', 'EViTS5_GLU', 'EViTS5_GLU']の逆順
            block_type = cfg.block_type[stage_id] \
                if isinstance(cfg.block_type, list) else cfg.block_type


            # rms3d
            norm = cfg.norm[stage_id] if isinstance(cfg.norm, list) else cfg.norm

            # silu
            act = cfg.act[stage_id] if isinstance(cfg.act, list) else cfg.act

            # ステージのメインブロックを構築し、ステージに追加
            stage.extend(
                build_stage_main(
                    width=width,
                    depth=depth,
                    block_type=block_type,
                    norm=norm,
                    act=act,
                    input_width=(
                        width if cfg.upsample_match_channel else cfg.width_list[min(stage_id + 1, num_stages - 1)]
                    ),
                    is_video=cfg.is_video,
                )
            )

            # ステージをOpSequentialでラップしてリストに追加
            self.stages.insert(0, OpSequential(stage))

        # ステージのリストをnn.ModuleListに変換
        self.stages = nn.ModuleList(self.stages)

        # 4) 出力プロジェクションブロックを作成

        self.project_out = build_decoder_project_out_block(
            in_channels=cfg.width_list[0] if cfg.depth_list[0] > 0 else cfg.width_list[1],
            out_channels=cfg.in_channels,
            factor=1 if cfg.depth_list[0] > 0 else 2,
            upsample_block_type=cfg.upsample_block_type,
            norm=cfg.out_norm,
            act=cfg.out_act,
            is_video=cfg.is_video,
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        logger.info(f"Decoder.forward {x.shape=}")

        # 1) 入力プロジェクションブロックを通過

        # 自動勾配チェックポイントを使用してメモリ使用量を削減
        x = auto_grad_checkpoint(self.project_in, x)
        logger.debug(f"Decoder.forward after project_in {x.shape=}")

        # 2) 6つのステージを逆順に通過
        for i, stage in enumerate(reversed(self.stages)):

            if len(stage.op_list) == 0:
                continue

            # x = stage(x)

            # 自動勾配チェックポイントを使用してメモリ使用量を削減
            x = auto_grad_checkpoint(stage, x)
            logger.debug(f"Decoder.forward after stage {i=} {x.shape=}")

        # 3) 出力プロジェクションブロックを通過

        # Discriminatorを用いた学習時の場合
        # False
        if self.disc_off_grad_ckpt:
            x = self.project_out(x)
        else:
            x = auto_grad_checkpoint(self.project_out, x)
        return x

### DC-AE

In [ ]:
@dataclass
class DCAEConfig:
    in_channels: int = 3
    latent_channels: int = 32
    time_compression_ratio: int = 1
    spatial_compression_ratio: int = 32
    encoder: EncoderConfig = field(
        default_factory=lambda: EncoderConfig(in_channels="${..in_channels}", latent_channels="${..latent_channels}")
    )
    decoder: DecoderConfig = field(
        default_factory=lambda: DecoderConfig(in_channels="${..in_channels}", latent_channels="${..latent_channels}")
    )
    use_quant_conv: bool = False

    pretrained_path: Optional[str] = None
    pretrained_source: str = "dc-ae"

    scaling_factor: Optional[float] = None
    is_image_model: bool = False

    is_training: bool = False  # NOTE: set to True in vae train config

    use_spatial_tiling: bool = False
    use_temporal_tiling: bool = False
    spatial_tile_size: int = 256
    temporal_tile_size: int = 32
    tile_overlap_factor: float = 0.25

In [ ]:
def init_modules(model: Union[nn.Module, list[nn.Module]], init_type="trunc_normal") -> None:
    """
    モデルの重みを初期化する
    1回だけ使用

    Args:
        model (nn.Module or list[nn.Module]): 初期化するモデルまたはモデルのリスト
        init_type (str, optional): 初期化の種類 ("trunc_normal@std" または "normal@std")
    """
    logger.info(f"init_modules {model=} {init_type=}")

    _DEFAULT_INIT_PARAM = {"trunc_normal": 0.02}

    if isinstance(model, list):
        for sub_module in model:
            init_modules(sub_module, init_type)
    else:
        init_params = init_type.split("@")
        init_params = float(init_params[1]) if len(init_params) > 1 else None

        if init_type.startswith("trunc_normal"):
            init_func = lambda param: nn.init.trunc_normal_(
                param, std=(_DEFAULT_INIT_PARAM["trunc_normal"] if init_params is None else init_params)
            )
        elif init_type.startswith("normal"):
            init_func = lambda param: nn.init.normal_(
                param, std=(_DEFAULT_INIT_PARAM["trunc_normal"] if init_params is None else init_params)
            )
        else:
            raise NotImplementedError

        for m in model.modules():
            if isinstance(m, (nn.Conv2d, nn.Linear, nn.ConvTranspose2d)):
                init_func(m.weight)
                if m.bias is not None:
                    m.bias.data.zero_()
            elif isinstance(m, nn.Embedding):
                init_func(m.weight)
            elif isinstance(m, (_BatchNorm, nn.GroupNorm, nn.LayerNorm)):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            else:
                weight = getattr(m, "weight", None)
                bias = getattr(m, "bias", None)
                if isinstance(weight, torch.nn.Parameter):
                    init_func(weight)
                if isinstance(bias, torch.nn.Parameter):
                    bias.data.zero_()

In [ ]:
class DCAE(nn.Module):
    """
    Deep Compressive Autoencoder (DCAE)
    """

    def __init__(self, cfg: DCAEConfig):
        """
        Args:
            cfg (DCAEConfig): DCAEの設定
        """
        logger.info(f"DCAE.__init__ {cfg=}")
        super().__init__()
        self.cfg = cfg

        # エンコーダを初期化
        self.encoder = Encoder(cfg.encoder)

        # デコーダを初期化
        self.decoder = Decoder(cfg.decoder)

        self.scaling_factor = cfg.scaling_factor
        self.time_compression_ratio = cfg.time_compression_ratio
        self.spatial_compression_ratio = cfg.spatial_compression_ratio
        self.use_spatial_tiling = cfg.use_spatial_tiling
        self.use_temporal_tiling = cfg.use_temporal_tiling
        self.spatial_tile_size = cfg.spatial_tile_size
        self.temporal_tile_size = cfg.temporal_tile_size
        assert (
            cfg.spatial_tile_size // cfg.spatial_compression_ratio
        ), f"spatial tile size {cfg.spatial_tile_size} must be divisible by spatial compression of {cfg.spatial_compression_ratio}"
        self.spatial_tile_latent_size = cfg.spatial_tile_size // cfg.spatial_compression_ratio
        assert (
            cfg.temporal_tile_size // cfg.time_compression_ratio
        ), f"temporal tile size {cfg.temporal_tile_size} must be divisible by temporal compression of {cfg.time_compression_ratio}"
        self.temporal_tile_latent_size = cfg.temporal_tile_size // cfg.time_compression_ratio
        self.tile_overlap_factor = cfg.tile_overlap_factor
        if self.cfg.pretrained_path is not None:
            self.load_model()

        self.to(torch.float32)
        init_modules(self, init_type="trunc_normal")

    def load_model(self):
        if self.cfg.pretrained_source == "dc-ae":
            state_dict = torch.load(self.cfg.pretrained_path, map_location="cpu", weights_only=True)["state_dict"]
            self.load_state_dict(state_dict)
        else:
            raise NotImplementedError

    def get_last_layer(self):
        return self.decoder.project_out.op_list[2].conv.weight

    # @property
    # def spatial_compression_ratio(self) -> int:
    #     return 2 ** (self.decoder.num_stages - 1)

    def encode_single(self, x: torch.Tensor, is_video_encoder: bool = False) -> torch.Tensor:
        """
        単一サンプルのエンコードを実行
        encodeメソッドから呼び出される

        Args:
            x (torch.Tensor): 入力テンソル (1, C, H, W) または (1, C, F, H, W)
            is_video_encoder (bool, optional): エンコーダが動画用かどうか
        Returns:
            torch.Tensor: 潜在表現テンソル
        """
        logger.info(f"DCAE.encode_single {x.shape=} {is_video_encoder=}")

        # (1, 3, 4, 256, 256)

        # バッチサイズが1であることを確認
        assert x.shape[0] == 1

        # 入力が動画かどうかを確認
        # True
        is_video = x.dim() == 5

        # エンコーダが画像用の場合
        # False
        if is_video and not is_video_encoder:
            b, c, f, h, w = x.shape
            x = x.permute(0, 2, 1, 3, 4).reshape(-1, c, h, w)

        # エンコーダーでエンコードを実行
        z = self.encoder(x)

        # エンコーダーが画像用の場合
        # False
        if is_video and not is_video_encoder:
            z = z.unsqueeze(dim=0).permute(0, 2, 1, 3, 4)

        # スケーリングファクターが設定されている場合
        # None
        if self.scaling_factor is not None:
            # スケーリングを適用
            z = z / self.scaling_factor

        return z

    def _encode(self, x: torch.Tensor) -> torch.Tensor:
        """
        エンコードの内部処理
        encodeメソッドから呼び出される

        Args:
            x (torch.Tensor): 入力テンソル
        Returns:
            torch.Tensor: 潜在表現テンソル
        """
        logger.info(f"DCAE._encode {x.shape=}")

        # (1, 3, 4, 256, 256)

        # 訓練時はそのままエンコード
        # False
        if self.cfg.is_training:
            return self.encoder(x)

        # 入力が動画か
        # True
        is_video_encoder = self.encoder.cfg.is_video if self.encoder.cfg.is_video is not None else False

        x_ret = []

        # バッチごとにループ（1）
        for i in range(x.shape[0]):

            # 単一サンプルのエンコードを実行
            x_ret.append(self.encode_single(x[i : i + 1], is_video_encoder))

        # 結果を結合して返す
        return torch.cat(x_ret, dim=0)

    def blend_v(self, a: torch.Tensor, b: torch.Tensor, blend_extent: int) -> torch.Tensor:
        blend_extent = min(a.shape[-2], b.shape[-2], blend_extent)
        for y in range(blend_extent):
            b[:, :, :, y, :] = a[:, :, :, -blend_extent + y, :] * (1 - y / blend_extent) + b[:, :, :, y, :] * (
                y / blend_extent
            )
        return b

    def blend_h(self, a: torch.Tensor, b: torch.Tensor, blend_extent: int) -> torch.Tensor:
        blend_extent = min(a.shape[-1], b.shape[-1], blend_extent)
        for x in range(blend_extent):
            b[:, :, :, :, x] = a[:, :, :, :, -blend_extent + x] * (1 - x / blend_extent) + b[:, :, :, :, x] * (
                x / blend_extent
            )
        return b

    def blend_t(self, a: torch.Tensor, b: torch.Tensor, blend_extent: int) -> torch.Tensor:
        blend_extent = min(a.shape[-3], b.shape[-3], blend_extent)
        for x in range(blend_extent):
            b[:, :, x, :, :] = a[:, :, -blend_extent + x, :, :] * (1 - x / blend_extent) + b[:, :, x, :, :] * (
                x / blend_extent
            )
        return b

    def spatial_tiled_encode(self, x: torch.Tensor) -> torch.Tensor:
        net_size = int(self.spatial_tile_size * (1 - self.tile_overlap_factor))
        blend_extent = int(self.spatial_tile_latent_size * self.tile_overlap_factor)
        row_limit = self.spatial_tile_latent_size - blend_extent

        # Split video into tiles and encode them separately.
        rows = []
        for i in range(0, x.shape[-2], net_size):
            row = []
            for j in range(0, x.shape[-1], net_size):
                tile = x[:, :, :, i : i + self.spatial_tile_size, j : j + self.spatial_tile_size]
                tile = self._encode(tile)
                row.append(tile)
            rows.append(row)
        result_rows = []
        for i, row in enumerate(rows):
            result_row = []
            for j, tile in enumerate(row):
                # blend the above tile and the left tile
                # to the current tile and add the current tile to the result row
                if i > 0:
                    tile = self.blend_v(rows[i - 1][j], tile, blend_extent)
                if j > 0:
                    tile = self.blend_h(row[j - 1], tile, blend_extent)
                result_row.append(tile[:, :, :, :row_limit, :row_limit])
            result_rows.append(torch.cat(result_row, dim=-1))

        return torch.cat(result_rows, dim=-2)

    def temporal_tiled_encode(self, x: torch.Tensor) -> torch.Tensor:
        overlap_size = int(self.temporal_tile_size * (1 - self.tile_overlap_factor))
        blend_extent = int(self.temporal_tile_latent_size * self.tile_overlap_factor)
        t_limit = self.temporal_tile_latent_size - blend_extent

        # Split the video into tiles and encode them separately.
        row = []
        for i in range(0, x.shape[2], overlap_size):
            tile = x[:, :, i : i + self.temporal_tile_size, :, :]
            if self.use_spatial_tiling and (
                tile.shape[-1] > self.spatial_tile_size or tile.shape[-2] > self.spatial_tile_size
            ):
                tile = self.spatial_tiled_encode(tile)
            else:
                tile = self._encode(tile)
            row.append(tile)
        result_row = []
        for i, tile in enumerate(row):
            if i > 0:
                tile = self.blend_t(row[i - 1], tile, blend_extent)
            result_row.append(tile[:, :, :t_limit, :, :])

        return torch.cat(result_row, dim=2)

    def encode(self, x: torch.Tensor) -> torch.Tensor:
        """
        DCAEのエンコード処理

        Args:
            x (torch.Tensor): 入力テンソル
        Returns:
            torch.Tensor: 潜在表現テンソル
        """
        logger.info(f"DCAE.encode {x.shape=}")

        # 時間方向にタイル分割する場合（16フレームより大きい）
        if self.use_temporal_tiling and x.shape[2] > self.temporal_tile_size:
            return self.temporal_tiled_encode(x)

        # 空間方向にタイル分割する場合（256ピクセルより大きい）
        elif self.use_spatial_tiling and (x.shape[-1] > self.spatial_tile_size or x.shape[-2] > self.spatial_tile_size):

            return self.spatial_tiled_encode(x)

        # タイル処理しない場合
        # True
        else:
            return self._encode(x)

    def spatial_tiled_decode(self, z: torch.FloatTensor) -> torch.Tensor:
        net_size = int(self.spatial_tile_latent_size * (1 - self.tile_overlap_factor))
        blend_extent = int(self.spatial_tile_size * self.tile_overlap_factor)
        row_limit = self.spatial_tile_size - blend_extent

        # Split z into overlapping tiles and decode them separately.
        # The tiles have an overlap to avoid seams between tiles.
        rows = []
        for i in range(0, z.shape[-2], net_size):
            row = []
            for j in range(0, z.shape[-1], net_size):
                tile = z[:, :, :, i : i + self.spatial_tile_latent_size, j : j + self.spatial_tile_latent_size]
                decoded = self._decode(tile)
                row.append(decoded)
            rows.append(row)
        result_rows = []
        for i, row in enumerate(rows):
            result_row = []
            for j, tile in enumerate(row):
                # blend the above tile and the left tile
                # to the current tile and add the current tile to the result row
                if i > 0:
                    tile = self.blend_v(rows[i - 1][j], tile, blend_extent)
                if j > 0:
                    tile = self.blend_h(row[j - 1], tile, blend_extent)
                result_row.append(tile[:, :, :, :row_limit, :row_limit])
            result_rows.append(torch.cat(result_row, dim=-1))

        return torch.cat(result_rows, dim=-2)

    def temporal_tiled_decode(self, z: torch.Tensor) -> torch.Tensor:
        overlap_size = int(self.temporal_tile_latent_size * (1 - self.tile_overlap_factor))
        blend_extent = int(self.temporal_tile_size * self.tile_overlap_factor)
        t_limit = self.temporal_tile_size - blend_extent

        row = []
        for i in range(0, z.shape[2], overlap_size):
            tile = z[:, :, i : i + self.temporal_tile_latent_size, :, :]
            if self.use_spatial_tiling and (
                tile.shape[-1] > self.spatial_tile_latent_size or tile.shape[-2] > self.spatial_tile_latent_size
            ):
                decoded = self.spatial_tiled_decode(tile)
            else:
                decoded = self._decode(tile)
            row.append(decoded)
        result_row = []
        for i, tile in enumerate(row):
            if i > 0:
                tile = self.blend_t(row[i - 1], tile, blend_extent)
            result_row.append(tile[:, :, :t_limit, :, :])

        return torch.cat(result_row, dim=2)

    def decode_single(self, z: torch.Tensor, is_video_decoder: bool = False) -> torch.Tensor:
        """
        単一サンプルのデコードを実行
        _decodeメソッドから呼び出される

        Args:
            z (torch.Tensor): 潜在表現テンソル (1, C, H, W) または (1, C, F, H, W)
            is_video_decoder (bool, optional): デコーダが動画用かどうか
        Returns:
            torch.Tensor: 復元されたテンソル
        """
        logger.info(f"DCAE.decode_single {z.shape=} {is_video_decoder=}")

        # バッチサイズが1であることを確認
        assert z.shape[0] == 1

        # 入力が動画かどうかを確認
        # True
        is_video = z.dim() == 5

        # デコーダが画像用の場合
        # False
        if is_video and not is_video_decoder:
            b, c, f, h, w = z.shape
            z = z.permute(0, 2, 1, 3, 4).reshape(-1, c, h, w)

        # スケーリングファクターが設定されている場合
        # None
        if self.scaling_factor is not None:
            # スケーリングを適用
            z = z * self.scaling_factor

        # デコーダーでデコードを実行
        x = self.decoder(z)

        # デコーダーが画像用の場合
        # False
        if is_video and not is_video_decoder:
            x = x.unsqueeze(dim=0).permute(0, 2, 1, 3, 4)

        return x

    def _decode(self, z: torch.Tensor) -> torch.Tensor:
        """
        デコードの内部処理
        decodeメソッドから呼び出される

        Args:
            z (torch.Tensor): 潜在表現テンソル
        """
        logger.info(f"DCAE._decode {z.shape=}")

        # 訓練時はそのままデコード
        # False
        if self.cfg.is_training:
            return self.decoder(z)

        # 入力が動画か
        # True
        is_video_decoder = self.decoder.cfg.is_video \
            if self.decoder.cfg.is_video is not None else False


        x_ret = []

        # バッチごとにループ（1）
        for i in range(z.shape[0]):
            x_ret.append(self.decode_single(z[i : i + 1], is_video_decoder))

        return torch.cat(x_ret, dim=0)

    def decode(self, z: torch.Tensor) -> torch.Tensor:
        """
        DCAEのデコード処理
        forwardメソッドから呼び出される

        Args:
            z (torch.Tensor): 潜在表現テンソル
        """
        logger.info(f"DCAE.decode {z.shape=}")

        # 時間方向にタイル分割する場合
        # False
        if self.use_temporal_tiling and z.shape[2] > self.temporal_tile_latent_size:
            return self.temporal_tiled_decode(z)

        # 空間方向にタイル分割する場合
        # False
        elif self.use_spatial_tiling and (
            z.shape[-1] > self.spatial_tile_latent_size or z.shape[-2] > self.spatial_tile_latent_size
        ):
            return self.spatial_tiled_decode(z)

        # タイル処理しない場合
        # True
        else:
            return self._decode(z)

    def forward(self, x: torch.Tensor) -> tuple[Any, Tensor, dict[Any, Any]]:
        """
        DCAEの順伝搬
        DC-AEのエントリーポイント

        Args:
            x (torch.Tensor): 入力テンソル
        Returns:
            tuple[Any, Tensor, dict[Any, Any]]:
                - torch.Tensor: 復元されたテンソル
                - Tensor: None (量子化情報用のプレースホルダー)
                - dict[Any, Any]: 潜在表現
        """
        logger.info(f"DCAE.forward {x.shape=}")

        # 元のデータ型を保存
        x_type = x.dtype

        # 画像入力かを取得
        # False
        is_image_model = self.cfg.__dict__.get("is_image_model", False)

        # エンコーダの重みのデータ型に変換
        x = x.to(self.encoder.project_in.conv.weight.dtype)

        # 画像入力の場合
        # False
        if is_image_model:
            b, c, _, h, w = x.shape
            x = x.permute(0, 2, 1, 3, 4).reshape(-1, c, h, w)

        # エンコード
        z = self.encode(x)

        # デコード
        dec = self.decode(z)

        # 画像入力の場合
        # False
        if is_image_model:
            dec = dec.reshape(b, 1, c, h, w).permute(0, 2, 1, 3, 4)
            z = z.unsqueeze(dim=0).permute(0, 2, 1, 3, 4)

        # 元のデータ型に戻す
        dec = dec.to(x_type)
        return dec, None, z

    def get_latent_size(self, input_size: list[int]) -> list[int]:
        latent_size = []
        # T
        latent_size.append((input_size[0] - 1) // self.time_compression_ratio + 1)
        # H, w
        for i in range(1, 3):
            latent_size.append((input_size[i] - 1) // self.spatial_compression_ratio + 1)
        return latent_size

### HuggingFace Hub対応DC-AE

In [ ]:
def dc_ae_f32(name: str, pretrained_path: str) -> DCAEConfig:
    """
    DCAEの設定を初期化する

    Args:
        name (str): モデルの名前
        pretrained_path (str): 事前学習済みモデルのパス
    Returns:
        DCAEConfig: DCAEの設定
    """

    if name in ["dc-ae-f32t4c128"]:
        cfg_str = (
            "time_compression_ratio=4 "
            "spatial_compression_ratio=32 "
            "encoder.block_type=[ResBlock,ResBlock,ResBlock,EViTS5_GLU,EViTS5_GLU,EViTS5_GLU] "
            "encoder.width_list=[128,256,512,512,1024,1024] encoder.depth_list=[2,2,2,3,3,3] "
            "encoder.downsample_block_type=Conv "
            "encoder.norm=rms3d "
            "encoder.is_video=True "
            "decoder.block_type=[ResBlock,ResBlock,ResBlock,EViTS5_GLU,EViTS5_GLU,EViTS5_GLU] "
            "decoder.width_list=[128,256,512,512,1024,1024] decoder.depth_list=[3,3,3,3,3,3] "
            "decoder.upsample_block_type=InterpolateConv "
            "decoder.norm=rms3d decoder.act=silu decoder.out_norm=rms3d "
            "decoder.is_video=True "
            "encoder.temporal_downsample=[False,False,False,True,True,False] "
            "decoder.temporal_upsample=[False,False,False,True,True,False] "
            "latent_channels=128"
        )  # make sure there is no trailing blankspace in the last line
    else:
        raise NotImplementedError

    # 文字列から設定を読み込む
    cfg = OmegaConf.from_dotlist(cfg_str.split(" "))

    # DCAEConfigに流し込む
    cfg: DCAEConfig = OmegaConf.to_object(
        OmegaConf.merge(OmegaConf.structured(DCAEConfig), cfg)
    )

    # 事前学習済みモデルのパスを設定する
    cfg.pretrained_path = pretrained_path

    return cfg

In [ ]:
# 利用可能なDCAEモデルの定義
REGISTERED_DCAE_MODEL: dict[str, tuple[Callable, Optional[str]]] = {
    "dc-ae-f32t4c128": (dc_ae_f32, None),
}

def create_dc_ae_model_cfg(name: str, pretrained_path: Optional[str] = None) -> DCAEConfig:
    """
    登録されたDCAEモデル設定を作成する

    Args:
        name (str): 登録されたDCAEモデルの名前
        pretrained_path (Optional[str], optional): 事前学習済みモデルのパス
    Returns:
        DCAEConfig: DCAEモデルの設定
    """
    logger.info(f"create_dc_ae_model_cfg {name=} {pretrained_path=}")

    assert name in REGISTERED_DCAE_MODEL, f"{name} is not supported"

    # 設定を作成する関数とデフォルトの事前学習済みモデルのパスを取得
    dc_ae_cls, default_pt_path = REGISTERED_DCAE_MODEL[name]

    pretrained_path = default_pt_path if pretrained_path is None else pretrained_path

    # DCAEモデルの設定を作成
    model_cfg = dc_ae_cls(name, pretrained_path)

    return model_cfg

In [ ]:
class DCAE_HF(DCAE, PyTorchModelHubMixin):
    """
    HuggingFace Hub対応DCAEモデル
    """
    def __init__(self, model_name: str):
        logger.info(f"DCAE_HF.__init__ {model_name=}")

        # 1) DCAEモデルの設定を作成
        # dc-ae-f32t4c128
        cfg = create_dc_ae_model_cfg(model_name)

        # 2) DCAEを初期化
        DCAE.__init__(self, cfg)

### 検証

In [ ]:
# @MODELS.register_module("dc_ae")
def DC_AE(
    model_name: str,
    device_map: str | torch.device = "cuda",
    torch_dtype: torch.dtype = torch.bfloat16,
    from_scratch: bool = False,
    from_pretrained: str | None = None,
    is_training: bool = False,
    use_spatial_tiling: bool = False,
    use_temporal_tiling: bool = False,
    spatial_tile_size: int = 256,
    temporal_tile_size: int = 32,
    tile_overlap_factor: float = 0.25,
    scaling_factor: float = None,
    disc_off_grad_ckpt: bool = False,
) -> DCAE_HF:
    """
    Deep Compressive Autoencoder (DCAE)モデルを作成する

    Args:
        model_name (str): モデルの名前
        device_map (str or torch.device, optional): モデルを配置するデバイス
        torch_dtype (torch.dtype, optional): モデルのデータ型
        from_scratch (bool, optional): ランダム初期化からモデルを作成するかどうか
        from_pretrained (str or None, optional): 事前学習済みモデルのパス
        is_training (bool, optional): モデルをトレーニングモードに設定するかどうか
        use_spatial_tiling (bool, optional): 空間タイル処理を使用するかどうか
        use_temporal_tiling (bool, optional): 時間タイル処理を使用するかどうか
        spatial_tile_size (int, optional): 空間タイルのサイズ
        temporal_tile_size (int, optional): 時間タイルのサイズ
        tile_overlap_factor (float, optional): タイルの重なり係数
        scaling_factor (float, optional): スケーリングファクター
        disc_off_grad_ckpt (bool, optional): 勾配チェックポイントを無効にするかどうか
    Returns:
        DCAE_HF: DCAEモデル
    """
    logger.info(f"DC_AE {model_name=} {device_map=} {torch_dtype=} {from_scratch=} {from_pretrained=} {is_training=} {use_spatial_tiling=} {use_temporal_tiling=} {spatial_tile_size=} {temporal_tile_size=} {tile_overlap_factor=} {scaling_factor=} {disc_off_grad_ckpt=}")

    # False
    if not from_scratch:
        model = DCAE_HF.from_pretrained(model_name).to(device_map, torch_dtype)

    # True
    else:
        model = DCAE_HF(model_name).to(device_map, torch_dtype)

    # False
    if from_pretrained is not None:
        model = load_checkpoint(model, from_pretrained, device_map=device_map)
        print(f"loaded dc_ae from ckpt path: {from_pretrained}")

    model.cfg.is_training = is_training
    model.use_spatial_tiling = use_spatial_tiling
    model.use_temporal_tiling = use_temporal_tiling
    model.spatial_tile_size = spatial_tile_size
    model.temporal_tile_size = temporal_tile_size
    model.tile_overlap_factor = tile_overlap_factor

    if scaling_factor is not None:
        model.scaling_factor = scaling_factor

    model.decoder.disc_off_grad_ckpt = disc_off_grad_ckpt
    return model

In [ ]:
if True: 
    logger.info(f"DCAEを検証")
    # (batch_size, channels, frames, height, width)
    # (1, 3, 96, 512, 512)
    sample_input = torch.randn(1, 3, 4, 256, 256).to("cuda")

    # video_dc_ae.pyのデフォルト設定
    model = DC_AE(
        model_name="dc-ae-f32t4c128",
        from_scratch=True,
        from_pretrained=None,
        is_training=False,
        use_spatial_tiling=True,
        use_temporal_tiling=True,
        spatial_tile_size=256,
        temporal_tile_size=16,
        tile_overlap_factor=0.25,
    )
    output = model(sample_input)
    print(output[0].shape)  # Decoded output

## MMDiT

### LigerEmbedND

In [ ]:
def liger_rope(pos: Tensor, dim: int, theta: int) -> Tuple:
    """
    Liger RoPE
    Tritonで最適化されたliger-kernel
    LigerEmbedNDで使用

    Args:
        pos (Tensor): 位置エンコーディングの位置テンソル (..., n)
        dim (int): 埋め込み次元数
        theta (int): RoPEのスケーリングパラメータ
    Returns:
        Tuple: コサインとサインのテンソル (..., n, dim//2)
    """
    logger.info(f"liger_rope {pos.shape=} {dim=} {theta=}")

    assert dim % 2 == 0

    # 1) 逆周波数を計算

    scale = torch.arange(
        0,
        dim,
        2,
        dtype=torch.float32,
        device=pos.device
    ) / dim

    omega = 1.0 / (theta**scale)

    out = torch.einsum("...n,d->...nd", pos, omega)  # (b, seq, dim//2)

    # 2) コサインとサインを計算

    cos = out.cos()
    sin = out.sin()

    return (cos, sin)

In [ ]:
class LigerEmbedND(nn.Module):
    """
    Liger Multi-dimensional RoPE Embedding
    MMDiTModelで使用
    """

    def __init__(self, dim: int, theta: int, axes_dim: list[int]):
        """
        Args:
            dim (int): 埋め込み次元数
            theta (int): RoPEのスケーリングパラメータ
            axes_dim (list[int]): 各軸の次元数
        """
        logger.info(f"LigerEmbedND.__init__ {dim=} {theta=} {axes_dim=}")
        super().__init__()
        self.dim = dim
        self.theta = theta
        self.axes_dim = axes_dim

    def forward(self, ids: Tensor) -> Tensor:
        logger.info(f"LigerEmbedND.forward {ids.shape=}")
        n_axes = ids.shape[-1]
        cos_list = []
        sin_list = []
        for i in range(n_axes):
            cos, sin = liger_rope(ids[..., i], self.axes_dim[i], self.theta)
            cos_list.append(cos)
            sin_list.append(sin)
        cos_emb = torch.cat(cos_list, dim=-1).repeat(1, 1, 2).contiguous()
        sin_emb = torch.cat(sin_list, dim=-1).repeat(1, 1, 2).contiguous()

        return (cos_emb, sin_emb)

### MLPEmbedder

In [ ]:
class MLPEmbedder(nn.Module):
    """
    MLP Embedder
    MMDiTModelで使用
    """
    def __init__(self, in_dim: int, hidden_dim: int):
        logger.info(f"MLPEmbedder.__init__ {in_dim=} {hidden_dim=}")
        super().__init__()
        self.in_layer = nn.Linear(in_dim, hidden_dim, bias=True)
        self.silu = nn.SiLU()
        self.out_layer = nn.Linear(hidden_dim, hidden_dim, bias=True)

    def forward(self, x: Tensor) -> Tensor:
        logger.info(f"MLPEmbedder.forward {x.shape=}")
        return self.out_layer(self.silu(self.in_layer(x)))

### QKNorm

In [ ]:
class RMSNorm(torch.nn.Module):
    """
    RMS Normalization
    FusedRMSNormで継承
    """

    def __init__(self, dim: int):
        """
        Args:
            dim (int): 埋め込み次元数
        """
        logger.info(f"RMSNorm.__init__ {dim=}")
        super().__init__()
        self.scale = nn.Parameter(torch.ones(dim))

    def forward(self, x: Tensor):
        logger.info(f"RMSNorm.forward {x.shape=}")
        x_dtype = x.dtype
        x = x.float()
        rrms = torch.rsqrt(torch.mean(x**2, dim=-1, keepdim=True) + 1e-6)
        return (x * rrms).to(dtype=x_dtype) * self.scale

In [ ]:
class FusedRMSNorm(RMSNorm):
    """
    Fused RMS Normalization
    QKNormで使用
    """

    def forward(self, x: Tensor):
        logger.info(f"FusedRMSNorm.forward {x.shape=}")
        return LigerRMSNormFunction.apply(
            x,
            self.scale,
            1e-6,
            0.0,
            "llama",
            False,
        )

In [ ]:
class QKNorm(torch.nn.Module):
    """
    Query-Key Normalization
    Self-Attentionで使用
    """

    def __init__(self, dim: int):
        """
        Args:
            dim (int): 埋め込み次元数
        """
        logger.info(f"QKNorm.__init__ {dim=}")
        super().__init__()
        self.query_norm = FusedRMSNorm(dim)
        self.key_norm = FusedRMSNorm(dim)

    def forward(self, q: Tensor, k: Tensor, v: Tensor) -> tuple[Tensor, Tensor]:
        """
        Args:
            q (Tensor): クエリテンソル (..., dim)
            k (Tensor): キーテンソル (..., dim)
            v (Tensor): バリューテンソル (..., dim)
        Returns:
            tuple[Tensor, Tensor]: 正規化されたクエリとキーのテンソル
        """
        logger.info(f"QKNorm.forward {q.shape=} {k.shape=} {v.shape=}")
        q = self.query_norm(q)
        k = self.key_norm(k)
        return q.to(v), k.to(v)

### アテンション機構

In [ ]:
def flash_attn_func(q: Tensor, k: Tensor, v: Tensor) -> Tensor:
    """
    Flash Attentionを使用したアテンション計算
    attention関数で使用

    Args:
        q (Tensor): クエリテンソル (B, H, L, D)
        k (Tensor): キーテンソル (B, H, L, D)
        v (Tensor): バリューテンソル (B, H, L, D)
    Returns:
        Tensor: アテンション出力テンソル (B, L, H, D)
    """
    logger.info(f"flash_attn_func {q.shape=} {k.shape=} {v.shape=}")

    if SUPPORT_FA3:
        return flash_attn_func_v3(q, k, v)[0]
    return flash_attn_func_v2(q, k, v)

In [ ]:
def attention(q: Tensor, k: Tensor, v: Tensor, pe) -> Tensor:
    """
    アテンション計算
    SingleStreamBlockProcessor, DoubleSreamBlockProcessor,
    SelfAttentionで使用

    Args:
        q (Tensor): クエリテンソル (B, L, H, D)
        k (Tensor): キーテンソル (B, L, H, D)
        v (Tensor): バリューテンソル (B, L, H, D)
        pe: 位置エンコーディングテンソルまたはコサイン・サインテンソルのタプル
    Returns:
        Tensor: アテンション出力テンソル (B, L, H*D)
    """
    logger.info(f"attention {q.shape=} {k.shape=} {v.shape=} {type(pe)=}")

    # 1) RoPEを適用

    if isinstance(pe, torch.Tensor):
        q, k = apply_rope(q, k, pe)
    else:
        cos, sin = pe
        logger.debug(f"LigerRopeを適用 {cos.shape=} {sin.shape=}")
        q, k = LigerRopeFunction.apply(q, k, cos, sin)

    # 2) Flash Attentionを計算

    q = rearrange(q, "B H L D -> B L H D")
    k = rearrange(k, "B H L D -> B L H D")
    v = rearrange(v, "B H L D -> B L H D")

    x = flash_attn_func(q, k, v)

    # 3) ヘッドを結合

    x = rearrange(x, "B L H D -> B L (H D)")

    logger.debug(f"attention output {x.shape=}")

    return x

### セルフアテンション

In [ ]:
class SelfAttention(nn.Module):
    """
    セルフアテンション
    DoubleStreamBlockで合計2回使用
    """

    def __init__(self, dim: int, num_heads: int = 8, qkv_bias: bool = False, fused_qkv: bool = True):
        """
        Args:
            dim (int): 埋め込み次元数
            num_heads (int, optional): アテンションヘッド数. Defaults to 8.
            qkv_bias (bool, optional): QKVのバイアスを使用するかどうか. Defaults to False.
            fused_qkv (bool, optional): QKVを融合した線形層を使用するかどうか. Defaults to True.
        """
        logger.info(f"SelfAttention.__init__ {dim=} {num_heads=} {qkv_bias=} {fused_qkv=}")

        # 1) 初期化

        super().__init__()
        self.num_heads = num_heads # 3
        self.fused_qkv = fused_qkv # False
        head_dim = dim // num_heads # 384 // 3 = 128

        # 2) QKVの線形層を定義

        # False
        if fused_qkv:
            self.qkv = nn.Linear(dim, dim * 3, bias=qkv_bias)

        # True
        else:
            self.q_proj = nn.Linear(dim, dim, bias=qkv_bias)
            self.k_proj = nn.Linear(dim, dim, bias=qkv_bias)
            self.v_proj = nn.Linear(dim, dim, bias=qkv_bias)

        # 3) QKNormを定義

        self.norm = QKNorm(head_dim)

        # 4) 出力の線形層を定義

        self.proj = nn.Linear(dim, dim)

    def forward(self, x: Tensor, pe: Tensor) -> Tensor:
        logger.info(f"SelfAttention.forward {x.shape=} {pe.shape=}")

        # 1) QKVを計算

        # False
        if self.fused_qkv:
            qkv = self.qkv(x)
            q, k, v = rearrange(qkv, "B L (K H D) -> K B H L D", K=3, H=self.num_heads)

        # True
        else:
            q = rearrange(self.q_proj(x), "B L (H D) -> B L H D", H=self.num_heads)
            k = rearrange(self.k_proj(x), "B L (H D) -> B L H D", H=self.num_heads)
            v = rearrange(self.v_proj(x), "B L (H D) -> B L H D", H=self.num_heads)
            logger.debug(f"Before QKNorm {q.shape=} {k.shape=} {v.shape=}")

        q, k = self.norm(q, k, v)

        # False
        if not self.fused_qkv:
            q = rearrange(q, "B L H D -> B H L D")
            k = rearrange(k, "B L H D -> B H L D")
            v = rearrange(v, "B L H D -> B H L D")
            logger.debug(f"After QKNorm {q.shape=} {k.shape=} {v.shape=}")

        # 2) アテンションを計算
        x = attention(q, k, v, pe=pe)

        # 3) 出力を計算
        x = self.proj(x)
        logger.debug(f"SelfAttention output {x.shape=}")

        return x

### Modulation

In [ ]:
@dataclass
class ModulationOut:
    shift: Tensor
    scale: Tensor
    gate: Tensor

In [ ]:
class Modulation(nn.Module):
    """
    モジュレーション層
    AdaLN層の変調（モジュレーション）で使用するシフト・スケール・ゲートを生成する
    DoubleStreamBlockで使用
    """

    def __init__(self, dim: int, double: bool):
        """
        Args:
            dim (int): 埋め込み次元数
            double (bool):
                ダブルモジュレーションを使用するかどうか
                ダブルモジュレーションの場合は、2セットのシフト・スケール・ゲートを生成する
        """
        logger.info(f"Modulation.__init__ {dim=} {double=}")
        super().__init__()

        # True or False
        self.is_double = double

        # 6 or 3
        self.multiplier = 6 if double else 3

        # 384 -> 384*6 or 384 -> 384*3
        self.lin = nn.Linear(dim, self.multiplier * dim, bias=True)

    def forward(self, vec: Tensor) -> tuple[ModulationOut, ModulationOut | None]:
        """
        Args:
            vec (Tensor): 入力テンソル (B, L, D)
        Returns:
            tuple[ModulationOut, ModulationOut | None]:
                モジュレーション出力（shift, scale, gate）のタプル
                ダブルモジュレーションの場合は2つ目のModulationOutも返す
        """
        logger.info(f"Modulation.forward {vec.shape=}")

        # (1, 384) -> (1, 384*6) or (1, 384) -> (1, 384*3)
        out = self.lin(nn.functional.silu(vec))[:, None, :].chunk(self.multiplier, dim=-1)
        logger.debug(f"Modulation.forward out shapes: {[o.shape for o in out]}")

        return (
            ModulationOut(*out[:3]),
            ModulationOut(*out[3:]) if self.is_double else None,
        )

### DoubleStreamBlock

In [ ]:
class DoubleStreamBlockProcessor:
    """
    ダブルストリームブロックプロセッサ
    DoubleStreamBlockで使用
    画像とテキストの両方のストリームを処理する
    """

    def __call__(self, attn: nn.Module, img: Tensor, txt: Tensor, vec: Tensor, pe: Tensor) -> tuple[Tensor, Tensor]:
        """
        動画とテキスト埋め込みを別々に処理する
        処理の過程でベクトルテンソルが両方に影響を与える

        Args:
            attn (nn.Module): ダブルストリームブロックモジュール
            img (Tensor): 動画テンソル (B, L_img, C)
            txt (Tensor): テキストテンソル (B, L_txt, C)
            vec (Tensor): ベクトルテンソル (B, C)
            pe (Tensor): 位置エンコーディングテンソルまたはコサイン・サインテンソルのタプル
        Returns:
            tuple[Tensor, Tensor]: 処理された動画テンソルとテキストテンソル
        """
        logger.info(f"DoubleStreamBlockProcessor.__call__ {img.shape=} {txt.shape=} {vec.shape=} {pe[0].shape=} {pe[1].shape=}")

        # 1) ベクトルテンソルからシフト・スケール・ゲートを取得

        img_mod1, img_mod2 = attn.img_mod(vec)
        txt_mod1, txt_mod2 = attn.txt_mod(vec)

        # 2) 動画テンソルにAdaLNを適用

        # レイヤー正規化を適用
        img_modulated = attn.img_norm1(img)

        # スケールとシフトで変調
        img_modulated = (1 + img_mod1.scale) * img_modulated + img_mod1.shift

        # False
        if attn.img_attn.fused_qkv:
            img_qkv = attn.img_attn.qkv(img_modulated)
            img_q, img_k, img_v = rearrange(img_qkv, "B L (K H D) -> K B H L D", K=3, H=attn.num_heads, D=attn.head_dim)
        else:

            # クエリを計算
            img_q = rearrange(
                attn.img_attn.q_proj(img_modulated),
                "B L (H D) -> B L H D",
                H=attn.num_heads
            )

            # キーを計算
            img_k = rearrange(
                attn.img_attn.k_proj(img_modulated),
                "B L (H D) -> B L H D",
                H=attn.num_heads
            )

            # バリューを計算
            img_v = rearrange(
                attn.img_attn.v_proj(img_modulated),
                "B L (H D) -> B L H D",
                H=attn.num_heads
            )

            logger.debug(f"{img_q.shape=} {img_k.shape=} {img_v.shape=}")


        # Stable Diffusion 3論文のQK正規化を適用
        img_q, img_k = attn.img_attn.norm(img_q, img_k, img_v)

        # True
        if not attn.img_attn.fused_qkv:
            img_q = rearrange(img_q, "B L H D -> B H L D")
            img_k = rearrange(img_k, "B L H D -> B H L D")
            img_v = rearrange(img_v, "B L H D -> B H L D")
            logger.debug(f"{img_q.shape=} {img_k.shape=} {img_v.shape=}")

        # 3) テキストテンソルにAdaLNを適用

        # レイヤー正規化を適用
        txt_modulated = attn.txt_norm1(txt)

        # スケールとシフトで変調
        txt_modulated = (1 + txt_mod1.scale) * txt_modulated + txt_mod1.shift

        # False
        if attn.txt_attn.fused_qkv:
            txt_qkv = attn.txt_attn.qkv(txt_modulated)
            txt_q, txt_k, txt_v = rearrange(txt_qkv, "B L (K H D) -> K B H L D", K=3, H=attn.num_heads, D=attn.head_dim)
        else:
            txt_q = rearrange(
                attn.txt_attn.q_proj(txt_modulated),
                "B L (H D) -> B L H D",
                H=attn.num_heads
            )

            txt_k = rearrange(
                attn.txt_attn.k_proj(txt_modulated),
                "B L (H D) -> B L H D",
                H=attn.num_heads
            )

            txt_v = rearrange(
                attn.txt_attn.v_proj(txt_modulated),
                "B L (H D) -> B L H D",
                H=attn.num_heads
            )
            logger.debug(f"Before QKNorm {txt_q.shape=} {txt_k.shape=} {txt_v.shape=}")

        # Stable Diffusion 3論文のQK正規化を適用
        txt_q, txt_k = attn.txt_attn.norm(txt_q, txt_k, txt_v)

        # True
        if not attn.txt_attn.fused_qkv:
            txt_q = rearrange(txt_q, "B L H D -> B H L D")
            txt_k = rearrange(txt_k, "B L H D -> B H L D")
            txt_v = rearrange(txt_v, "B L H D -> B H L D")
            logger.debug(f"After QKNorm {txt_q.shape=} {txt_k.shape=} {txt_v.shape=}")

        # 4) アテンションを計算

        # 画像とテキストのクエリ、キー、バリューを連結

        q = torch.cat((txt_q, img_q), dim=2)
        k = torch.cat((txt_k, img_k), dim=2)
        v = torch.cat((txt_v, img_v), dim=2)

        logger.debug(f"Before Attention {q.shape=} {k.shape=} {v.shape=}")

        # アテンションを計算
        attn1 = attention(q, k, v, pe=pe)

        # 5) アテンション出力を画像とテキストに分割

        txt_attn, img_attn = attn1[:, : txt_q.shape[2]], attn1[:, txt_q.shape[2] :]

        # 6) 出力を計算 

        img = img + img_mod1.gate * attn.img_attn.proj(img_attn)
        img = img + img_mod2.gate * attn.img_mlp(
            (1 + img_mod2.scale) * attn.img_norm2(img) + img_mod2.shift
        )

        txt = txt + txt_mod1.gate * attn.txt_attn.proj(txt_attn)
        txt = txt + txt_mod2.gate * attn.txt_mlp(
            (1 + txt_mod2.scale) * attn.txt_norm2(txt) + txt_mod2.shift
        )

        return img, txt

In [ ]:
class DoubleStreamBlock(nn.Module):
    """
    ダブルストリームブロック
    MMDiTModelで1回使用
    画像とテキストの両方のストリームを処理する
    """

    def __init__(self, hidden_size: int, num_heads: int, mlp_ratio: float, qkv_bias: bool = False, fused_qkv: bool = True):
        """
        Args:
            hidden_size (int): 埋め込み次元数
            num_heads (int): アテンションヘッド数
            mlp_ratio (float): MLPの隠れ層の次元数の比率
            qkv_bias (bool, optional): QKVのバイアスを使用するかどうか
            fused_qkv (bool, optional): QKVを融合した線形層を使用するかどうか
        """
        logger.info(f"DoubleStreamBlock.__init__ {hidden_size=} {num_heads=} {mlp_ratio=} {qkv_bias=} {fused_qkv=}")

        # 1) 初期化

        super().__init__()

        # 384 * 4 = 1536
        mlp_hidden_dim = int(hidden_size * mlp_ratio)

        # 3
        self.num_heads = num_heads

        # 384
        self.hidden_size = hidden_size

        # 384 // 3 = 128
        self.head_dim = hidden_size // num_heads

        # 2) 動画ストリームを定義

        # image stream

        # AdaLN用のモジュレーション層
        self.img_mod = Modulation(hidden_size, double=True)

        # AdaLN用の前半のレイヤー正規化層    
        self.img_norm1 = nn.LayerNorm(hidden_size, elementwise_affine=False, eps=1e-6)

        # セルフアテンション層
        self.img_attn = SelfAttention(
            dim=hidden_size, # 384
            num_heads=num_heads, # 3
            qkv_bias=qkv_bias, # True
            fused_qkv=fused_qkv # False
        )

        # AdaLN用の後半のレイヤー正規化層
        self.img_norm2 = nn.LayerNorm(hidden_size, elementwise_affine=False, eps=1e-6)

        # MLP層
        self.img_mlp = nn.Sequential(
            nn.Linear(hidden_size, mlp_hidden_dim, bias=True),
            nn.GELU(approximate="tanh"),
            nn.Linear(mlp_hidden_dim, hidden_size, bias=True),
        )

        # 3) テキストストリームを定義

        # AdaLN用のモジュレーション層
        self.txt_mod = Modulation(hidden_size, double=True)

        # AdaLN用の前半のレイヤー正規化層
        self.txt_norm1 = nn.LayerNorm(hidden_size, elementwise_affine=False, eps=1e-6)

        # セルフアテンション層
        self.txt_attn = SelfAttention(
            dim=hidden_size,
            num_heads=num_heads,
            qkv_bias=qkv_bias,
            fused_qkv=fused_qkv
        )

        # AdaLN用の後半のレイヤー正規化層
        self.txt_norm2 = nn.LayerNorm(hidden_size, elementwise_affine=False, eps=1e-6)

        # MLP層
        self.txt_mlp = nn.Sequential(
            nn.Linear(hidden_size, mlp_hidden_dim, bias=True),
            nn.GELU(approximate="tanh"),
            nn.Linear(mlp_hidden_dim, hidden_size, bias=True),
        )

        # 4) 動画とテキストの両方のストリームを処理するプロセッサを作成

        processor = DoubleStreamBlockProcessor()
        self.set_processor(processor)

    def set_processor(self, processor) -> None:
        self.processor = processor

    def get_processor(self):
        return self.processor

    def forward(self, img: Tensor, txt: Tensor, vec: Tensor, pe: Tensor, **kwargs) -> tuple[Tensor, Tensor]:
        """
        動画とテキスト埋め込みを別々に処理する
        処理の過程でベクトルテンソルが両方に影響を与える

        Args:
            img (Tensor): 動画テンソル (B, L_img, C)
            txt (Tensor): テキストテンソル (B, L_txt, C)
            vec (Tensor): ベクトルテンソル (B, C)
            pe (Tensor): 位置エンコーディングテンソルまたはコサイン・サインテンソルのタプル
        Returns:
            tuple[Tensor, Tensor]: 処理された動画テンソルとテキストテンソル
        """

        logger.info(f"DoubleStreamBlock.forward {img.shape=} {txt.shape=} {vec.shape=} {pe[0].shape=} {pe[1].shape=}")
        return self.processor(self, img, txt, vec, pe)

### SingleStreamBlock

In [ ]:
class SingleStreamBlockProcessor:
    """
    シングルストリームブロックプロセッサ
    SingleStreamBlockで使用
    """

    def __call__(self, attn: nn.Module, x: Tensor, vec: Tensor, pe: Tensor) -> Tensor:
        logger.info(f"SingleStreamBlockProcessor.__call__ {x.shape=} {vec.shape=} {pe[0].shape=} {pe[1].shape=}")

        # 1) AdaLNを適用

        # AdaLNのシフト・スケール・ゲートを作成する
        mod, _ = attn.modulation(vec)

        # レイヤー正規化を適用し、変調させる
        x_mod = (1 + mod.scale) * attn.pre_norm(x) + mod.shift

        # 2) クエリ、キー、バリュー、MLPを計算

        # False
        if attn.fused_qkv:
            qkv, mlp = torch.split(attn.linear1(x_mod), [3 * attn.hidden_size, attn.mlp_hidden_dim], dim=-1)
            q, k, v = rearrange(qkv, "B L (K H D) -> K B H L D", K=3, H=attn.num_heads)
        # True
        else:
            q = rearrange(attn.q_proj(x_mod), "B L (H D) -> B L H D", H=attn.num_heads)
            k = rearrange(attn.k_proj(x_mod), "B L (H D) -> B L H D", H=attn.num_heads)
            v, mlp = torch.split(attn.v_mlp(x_mod), [attn.hidden_size, attn.mlp_hidden_dim], dim=-1)
            logger.debug(f"Before QKNorm {q.shape=} {k.shape=} {v.shape=} {mlp.shape=}")

            v = rearrange(v, "B L (H D) -> B L H D", H=attn.num_heads)

        # 3) Stable Diffusion 3論文のQK正規化を適用
        q, k = attn.norm(q, k, v)

        # True
        if not attn.fused_qkv:
            q = rearrange(q, "B L H D -> B H L D")
            k = rearrange(k, "B L H D -> B H L D")
            v = rearrange(v, "B L H D -> B H L D")

            logger.debug(f"After QKNorm {q.shape=} {k.shape=} {v.shape=}")

        # 4) アテンションを計算

        attn_1 = attention(q, k, v, pe=pe)

        # 5) 出力を計算

        # MLPストリームで活性化を計算し、再度連結して2番目の線形層を実行
        output = attn.linear2(torch.cat((attn_1, attn.mlp_act(mlp)), 2))

        # シフト・スケール・ゲートで変調
        output = x + mod.gate * output

        return output

In [ ]:
class SingleStreamBlock(nn.Module):
    """
    シングルストリームブロック
    MMDiTModelで38個構築して使用

    並列線形層を持つDiTブロックで、変調インターフェースが適応
    https://arxiv.org/abs/2302.05442
    """

    def __init__(
        self,
        hidden_size: int,
        num_heads: int,
        mlp_ratio: float = 4.0,
        qk_scale: float | None = None,
        fused_qkv: bool = True,
    ):
        """
        Args:
            hidden_size (int): 埋め込み次元数
            num_heads (int): アテンションヘッド数
            mlp_ratio (float, optional): MLPの隠れ層の次元数の比率
            qk_scale (float | None, optional): QKスケーリングファクター
            fused_qkv (bool, optional): QKVを融合した線形層を使用するかどうか
        """
        logger.info(f"SingleStreamBlock.__init__ {hidden_size=} {num_heads=} {mlp_ratio=} {qk_scale=} {fused_qkv=}")

        # 1) 初期化

        super().__init__()
        self.hidden_dim = hidden_size # 384
        self.num_heads = num_heads # 3
        self.head_dim = hidden_size // num_heads # 384 // 3 = 128
        self.scale = qk_scale or self.head_dim**-0.5 # 128**-0.5
        self.fused_qkv = fused_qkv # False

        self.mlp_hidden_dim = int(hidden_size * mlp_ratio) # 384 * 4.0 = 1536

        # 2) クエリ・キー・バリューとMLPの線形層を構築

        # False
        if fused_qkv:
            # qkv and mlp_in
            self.linear1 = nn.Linear(hidden_size, hidden_size * 3 + self.mlp_hidden_dim)

        # True
        else:

            # 384 -> 384
            self.q_proj = nn.Linear(hidden_size, hidden_size)

            # 384 -> 384
            self.k_proj = nn.Linear(hidden_size, hidden_size)

            # 384 -> 384 + 1536
            self.v_mlp = nn.Linear(hidden_size, hidden_size + self.mlp_hidden_dim)

        # 3) アテンション出力とMLP出力を結合して最終線形層を構築

        # 384 + 1536 -> 384
        self.linear2 = nn.Linear(hidden_size + self.mlp_hidden_dim, hidden_size)

        # 4) QKNormを構築

        self.norm = QKNorm(self.head_dim)

        self.hidden_size = hidden_size

        # 5) レイヤー正規化層を構築

        self.pre_norm = nn.LayerNorm(hidden_size, elementwise_affine=False, eps=1e-6)

        # 6) GELU活性化関数を構築

        self.mlp_act = nn.GELU(approximate="tanh")

        # 7) モジュレーション層を構築

        self.modulation = Modulation(hidden_size, double=False)

        # 8) シングルストリームブロックプロセッサを作成

        processor = SingleStreamBlockProcessor()
        self.set_processor(processor)

    def set_processor(self, processor) -> None:
        self.processor = processor

    def get_processor(self):
        return self.processor

    def forward(self, x: Tensor, vec: Tensor, pe: Tensor, **kwargs) -> Tensor:
        logger.info(f"SingleStreamBlock.forward {x.shape=} {vec.shape=} {pe[0].shape=} {pe[1].shape=}")
        return self.processor(self, x, vec, pe)

### MMDiT

In [ ]:
@torch.compile(mode="max-autotune-no-cudagraphs", dynamic=True)
def timestep_embedding(t: Tensor, dim, max_period=10000, time_factor: float = 1000.0):
    """
    サイン波タイムステップ埋め込みを作成する

    Args:
        t (Tensor): バッチ要素ごとに1つのNインデックスを持つ1-Dテンソル。これらは分数値である可能性があります。
        dim (int): 出力の次元。
        max_period (int, optional): 埋め込みの最小周波数を制御します。デフォルトは10000。
        time_factor (float, optional): 時間スケーリングファクター。デフォルトは1000.0。
    Returns:
        Tensor: 埋め込みテンソルの形状は(t.shape[0], dim)
    """
    logger.info(f"timestep_embedding {t.shape=} {dim=} {max_period=} {time_factor=}")

    t = time_factor * t
    half = dim // 2
    freqs = torch.exp(-math.log(max_period) * torch.arange(start=0, end=half, dtype=torch.float32) / half).to(t.device)

    args = t[:, None].float() * freqs[None]
    embedding = torch.cat([torch.cos(args), torch.sin(args)], dim=-1)
    if dim % 2:
        embedding = torch.cat([embedding, torch.zeros_like(embedding[:, :1])], dim=-1)
    if torch.is_floating_point(t):
        embedding = embedding.to(t)
    return embedding

In [ ]:
class LastLayer(nn.Module):
    """
    DCAEデコーダの最後の層
    画像パッチを生成するために使用される
    """

    def __init__(self, hidden_size: int, patch_size: int, out_channels: int):
        """
        Args:
            hidden_size (int): 隠れ層の次元数
            patch_size (int): パッチのサイズ
            out_channels (int): 出力チャネル数
        """
        logger.info(f"LastLayer.__init__ {hidden_size=} {patch_size=} {out_channels=}")

        super().__init__()
        self.norm_final = nn.LayerNorm(hidden_size, elementwise_affine=False, eps=1e-6)
        self.linear = nn.Linear(hidden_size, patch_size * patch_size * out_channels, bias=True)
        self.adaLN_modulation = nn.Sequential(nn.SiLU(), nn.Linear(hidden_size, 2 * hidden_size, bias=True))

    def forward(self, x: Tensor, vec: Tensor) -> Tensor:
        """
        Args:
            x (Tensor): 入力テンソル (B, N, hidden_size)
            vec (Tensor): モジュレーションベクトル (B, hidden_size)
        Returns:
            Tensor: 出力テンソル (B, N, patch_size * patch_size * out_channels)
        """
        logger.info(f"LastLayer.forward {x.shape=} {vec.shape=}")
        shift, scale = self.adaLN_modulation(vec).chunk(2, dim=1)
        x = (1 + scale[:, None, :]) * self.norm_final(x) + shift[:, None, :]
        x = self.linear(x)
        return x


In [ ]:
@dataclass
class MMDiTConfig:
    model_type = "MMDiT"
    from_pretrained: str
    cache_dir: str
    in_channels: int
    vec_in_dim: int
    context_in_dim: int
    hidden_size: int
    mlp_ratio: float
    num_heads: int
    depth: int
    depth_single_blocks: int
    axes_dim: list[int]
    theta: int
    qkv_bias: bool
    guidance_embed: bool
    cond_embed: bool = False
    fused_qkv: bool = True
    grad_ckpt_settings: tuple[int, int] | None = None
    use_liger_rope: bool = False
    patch_size: int = 2

    def get(self, attribute_name, default=None):
        return getattr(self, attribute_name, default)

    def __contains__(self, attribute_name):
        return hasattr(self, attribute_name)

In [ ]:
class MMDiTModel(nn.Module):
    """
    マルチモーダルディフュージョントランスフォーマーモデル
    画像とテキストの両方のストリームを処理する
    Flux関数で使用

    """
    config_class = MMDiTConfig

    def __init__(self, config: MMDiTConfig):
        """
        Args:
            config (MMDiTConfig): モデルの設定
        """
        logger.info(f"MMDiTModel.__init__ {config=}")

        # 1) 初期化

        super().__init__()

        self.config = config

        self.in_channels = config.in_channels # 64

        self.out_channels = self.in_channels # 64

        self.patch_size = config.patch_size # 2

        # False
        # 384 % 3 = 0
        if config.hidden_size % config.num_heads != 0:
            raise ValueError(
                f"Hidden size {config.hidden_size} must be divisible by num_heads {config.num_heads}"
            )

        # 384 // 3 = 128
        pe_dim = config.hidden_size // config.num_heads

        # False
        # 16 + 56 + 56 = 128
        if sum(config.axes_dim) != pe_dim:
            raise ValueError(
                f"Got {config.axes_dim} but expected positional dim {pe_dim}"
            )

        # 384
        self.hidden_size = config.hidden_size

        # 3
        self.num_heads = config.num_heads

        # LigerEmbedND
        pe_embedder_cls = LigerEmbedND if config.use_liger_rope else EmbedND

        # 2) 位置埋め込み層を構築

        self.pe_embedder = pe_embedder_cls(
            dim=pe_dim, # 128
            theta=config.theta, # 10000
            axes_dim=config.axes_dim # [16, 56, 56]
        )

        # 3) 動画像の潜在変数の埋め込み層を構築

        # 64 -> 384
        self.img_in = nn.Linear(
            self.in_channels, # 64
            self.hidden_size, # 384
            bias=True
        )

        # 3) タイムステップ特徴量のMLP埋め込み層を構築

        # 256 -> 384
        self.time_in = MLPEmbedder(
            in_dim=256,
            hidden_dim=self.hidden_size # 384
        )

        # 4) グローバルテキスト特徴量（CLIP特徴量）のMLP埋め込み層を構築

        self.vector_in = MLPEmbedder(
            config.vec_in_dim, # 768
            self.hidden_size # 384
        )

        # 5) ガイダンス強度のMLP埋め込み層を構築

        self.guidance_in = (
            MLPEmbedder(in_dim=256, hidden_dim=self.hidden_size)
            if config.guidance_embed # False
            else nn.Identity()
        )

        # 6) I2Vの画像条件付き入力の埋め込み層を構築

        self.cond_in = (
            nn.Linear(
                self.in_channels + self.patch_size**2,
                self.hidden_size,
                bias=True
            )
            if config.cond_embed # False
            else nn.Identity()
        )

        # 7) 単語レベルのテキスト特徴量（T5特徴量）の埋め込み層を構築

        self.txt_in = nn.Linear(
            config.context_in_dim, # 4096
            self.hidden_size # 384
        )

        # 8) ダブルストリームブロックを構築(1個)

        self.double_blocks = nn.ModuleList(
            [
                DoubleStreamBlock(
                    self.hidden_size, # 384
                    self.num_heads, # 3
                    mlp_ratio=config.mlp_ratio, # 4.0
                    qkv_bias=config.qkv_bias, # True
                    fused_qkv=config.fused_qkv, # False
                )
                for _ in range(config.depth) # 1
            ]
        )

        # 9) シングルストリームブロックを構築(38個)

        self.single_blocks = nn.ModuleList(
            [
                SingleStreamBlock(
                    self.hidden_size, # 384
                    self.num_heads, # 3
                    mlp_ratio=config.mlp_ratio, # 4.0
                    fused_qkv=config.fused_qkv, # False
                )
                for _ in range(config.depth_single_blocks) # 38
            ]
        )

        # 10) 最終層を構築

        self.final_layer = LastLayer(
            self.hidden_size, # 384
            1,
            self.out_channels # 64
        )

        # 11) 重みを初期化

        self.initialize_weights()

        # 12) 順伝搬関数を設定

        # True
        if self.config.grad_ckpt_settings:
            self.forward = self.forward_selective_ckpt
        # False
        else:
            self.forward = self.forward_ckpt

        self._input_requires_grad = False

    def initialize_weights(self):
        if self.config.cond_embed:
            nn.init.zeros_(self.cond_in.weight)
            nn.init.zeros_(self.cond_in.bias)

    def prepare_block_inputs(
        self,
        img: Tensor,
        img_ids: Tensor,
        txt: Tensor,  # t5 encoded vec
        txt_ids: Tensor,
        timesteps: Tensor,
        y_vec: Tensor,  # clip encoded vec
        cond: Tensor = None,
        guidance: Tensor | None = None,
    ):
        """
        入力を各ブロックに適した形式に準備する

        Args:
            img (Tensor): 動画潜在変数テンソル (B, T, N, in_channels)
            img_ids (Tensor): 動画位置IDテンソル (B, T, N)
            txt (Tensor): テキストコンテキストテンソル (B, L, context_in_dim)
            txt_ids (Tensor): テキスト位置IDテンソル (B, L)
            timesteps (Tensor): タイムステップテンソル (B,)
            y_vec (Tensor): CLIPエンコードベクトルテンソル (B, vec_in_dim)
            cond (Tensor, optional): 条件画像テンソル (B, T, N, in_channels + patch_size**2)
            guidance (Tensor | None, optional): ガイダンス強度テンソル (B,)
        Returns:
            tuple[Tensor, Tensor, Tensor, Tensor]:
                ブロックに渡すために準備された入力テンソルのタプル
        """
        logger.info(f"MMDiTModel.prepare_block_inputs {img.shape=} {txt.shape=} {timesteps.shape=} {y_vec.shape=}")

        if img.ndim != 3 or txt.ndim != 3:
            raise ValueError("Input img and txt tensors must have 3 dimensions.")

        # 1) 動画の潜在変数に埋め込み層を適用

        img = self.img_in(img)
        logger.debug(f"After img_in: {img.shape=}")

        # 2) 条件画像の埋め込みを適用し、動画の潜在変数に加算

        # 条件画像がある場合
        # False
        if self.config.cond_embed:
            if cond is None:
                raise ValueError("Didn't get conditional input for conditional model.")

            # 条件画像の埋め込みを動画の潜在変数に加算
            img = img + self.cond_in(cond)

        # 3) タイムステップ特徴量に埋め込み層を適用

        vec = self.time_in(timestep_embedding(timesteps, 256))

        # 4) ガイダンス強度に埋め込み層を適用し、タイムステップ埋め込みに加算

        # ガイダンス強度の埋め込みを使用する場合
        # False
        if self.config.guidance_embed:
            if guidance is None:
                raise ValueError(
                    "Didn't get guidance strength for guidance distilled model."
                )

            # ガイダンス強度の埋め込みをタイムステップ埋め込みに加算 
            vec = vec + self.guidance_in(timestep_embedding(guidance, 256))

        # 5) CLIP特徴量に埋め込みを適用し、タイムステップ埋め込みに加算

        vec = vec + self.vector_in(y_vec)

        # 6) T5特徴量に埋め込みを適用

        txt = self.txt_in(txt)

        # 7) 位置エンコーディングに埋め込みを適用

        # テキストと画像の位置IDを連結
        # concat: 4096 + t*h*2/4
        ids = torch.cat((txt_ids, img_ids), dim=1)

        # 位置エンコーディングに埋め込みを適用
        pe = self.pe_embedder(ids)

        # False
        if self._input_requires_grad:
            # we only apply lora to double/single blocks, thus we only need to enable grad for these inputs
            img.requires_grad_()
            txt.requires_grad_()

        return img, txt, vec, pe

    def enable_input_require_grads(self):
        """Fit peft lora. This method should not be called manually."""
        self._input_requires_grad = True

    def forward_ckpt(
        self,
        img: Tensor,
        img_ids: Tensor,
        txt: Tensor,
        txt_ids: Tensor,
        timesteps: Tensor,
        y_vec: Tensor,
        cond: Tensor = None,
        guidance: Tensor | None = None,
        **kwargs,
    ) -> Tensor:

        # 1) 入力の特徴量を埋め込み、集約

        # 動画埋め込み, T5テキスト埋め込み, タイムステップ埋め込み, 位置エンコーディング
        # タイムステップ埋め込みは、CLIP特徴量とガイダンス強度を加算した値
        img, txt, vec, pe = self.prepare_block_inputs(
            img, img_ids, txt, txt_ids, timesteps, y_vec, cond, guidance
        )
        logger.debug(f"After prepare_block_inputs: {img.shape=} {txt.shape=} {vec.shape=} {pe[0].shape=} {pe[1].shape=}")

        # 2) ダブルストリームブロックを順伝搬

        # 1つのダブルストリームブロックを順伝搬
        for block in self.double_blocks:

            # 自動勾配チェックポイントを使用
            img, txt = auto_grad_checkpoint(block, img, txt, vec, pe)

        logger.debug(f"After double_blocks: {img.shape=} {txt.shape=}")

        # 3) テキスト埋め込みと画像埋め込みを連結

        img = torch.cat((txt, img), 1)
        logger.debug(f"After concat txt and img: {img.shape=}")

        # 4) シングルストリームブロックを順伝搬

        # 38個のシングルストリームブロックを順伝搬
        for block in self.single_blocks:

            # 自動勾配チェックポイントを使用
            img = auto_grad_checkpoint(block, img, vec, pe)

        logger.debug(f"After single_blocks: {img.shape=}")

        # 5) テキスト埋め込みを切り離し、画像埋め込みを取得

        img = img[:, txt.shape[1] :, ...]
        logger.debug(f"After single_blocks: {img.shape=}")

        # 6) 最終層を順伝搬し、画像パッチを生成

        img = self.final_layer(img, vec)  # (N, T, patch_size ** 2 * out_channels)

        logger.debug(f"After final_layer: {img.shape=}")

        return img

    def forward_selective_ckpt(
        self,
        img: Tensor,
        img_ids: Tensor,
        txt: Tensor,
        txt_ids: Tensor,
        timesteps: Tensor,
        y_vec: Tensor,
        cond: Tensor = None,
        guidance: Tensor | None = None,
        **kwargs,
    ) -> Tensor:
        logger.info(f"MMDiTModel.forward_selective_ckpt {img.shape=} {txt.shape=} {timesteps.shape=} {y_vec.shape=}")

        # 1) 入力の特徴量を埋め込み、集約

        # 動画埋め込み, T5テキスト埋め込み, タイムステップ埋め込み, 位置エンコーディング
        # タイムステップ埋め込みは、CLIP特徴量とガイダンス強度を加算した値
        img, txt, vec, pe = self.prepare_block_inputs(
            img, img_ids, txt, txt_ids, timesteps, y_vec, cond, guidance
        )
        logger.debug(f"After prepare_block_inputs: {img.shape=} {txt.shape=} {vec.shape=} {pe[0].shape=} {pe[1].shape=}")

        # 1) ダブルストリームブロックを順伝搬

        # 8
        ckpt_depth_double = self.config.grad_ckpt_settings[0]

        # 1つのダブルストリームブロックを順伝搬
        for block in self.double_blocks[:ckpt_depth_double]:
            img, txt = auto_grad_checkpoint(block, img, txt, vec, pe)

        for block in self.double_blocks[ckpt_depth_double:]:
            img, txt = block(img, txt, vec, pe)

        logger.debug(f"After double_blocks: {img.shape=} {txt.shape=}")

        # 2) テキスト埋め込みと画像埋め込みを連結

        img = torch.cat((txt, img), 1)
        logger.debug(f"After concat txt and img: {img.shape=}")

        # 3) シングルストリームブロックを順伝搬

        ckpt_depth_single = self.config.grad_ckpt_settings[1]

        for block in self.single_blocks[:ckpt_depth_single]:
            img = auto_grad_checkpoint(block, img, vec, pe)

        for block in self.single_blocks[ckpt_depth_single:]:
            img = block(img, vec, pe)

        logger.debug(f"After single_blocks: {img.shape=}")

        # 4) テキスト埋め込みを切り離し、画像埋め込みを取得

        img = img[:, txt.shape[1] :, ...]
        logger.debug(f"After single_blocks: {img.shape=}")

        # 5) 最終層を順伝搬し、画像パッチを生成

        img = self.final_layer(img, vec)  # (N, T, patch_size ** 2 * out_channels)
        logger.debug(f"After final_layer: {img.shape=}")


        return img

### 検証

In [ ]:
# @MODELS.register_module("flux")
def Flux(
    cache_dir: str = None,
    from_pretrained: str = None,
    device_map: str | torch.device = "cuda",
    torch_dtype: torch.dtype = torch.bfloat16,
    strict_load: bool = False,
    **kwargs,
) -> MMDiTModel:
    """
    MMDiTモデルを作成するファクトリ関数

    Args:
        cache_dir (str, optional): 事前学習済みモデルのキャッシュディレクトリ. Defaults to None.
        from_pretrained (str, optional): 事前学習済みモデルのパスまたは名前. Defaults to None.
        device_map (str | torch.device, optional): モデルを配置するデバイス. Defaults to "cuda".
        torch_dtype (torch.dtype, optional): モデルのデフォルトデータ型. Defaults to torch.bfloat16.
        strict_load (bool, optional): チェックポイントの厳密な読み込みを行うかどうか. Defaults to False.
    Returns:
        MMDiTModel: 初期化されたMMDiTモデル
    """
    logger.info(f"Initializing MMDiT Model with {from_pretrained=}, {cache_dir=}, {device_map=}, {torch_dtype=}, {strict_load=}, {kwargs=}")

    # 1) モデル設定を作成

    config = MMDiTConfig(
        from_pretrained=from_pretrained, # None
        cache_dir=cache_dir, # None
        **kwargs,
    )

    # False
    low_precision_init = from_pretrained is not None and len(from_pretrained) > 0

    # False
    if low_precision_init:
        default_dtype = torch.get_default_dtype()
        torch.set_default_dtype(torch_dtype)

    # 2) モデルをデバイスに配置して初期化

    with torch.device(device_map):
        model = MMDiTModel(config)

    # False
    if low_precision_init:
        torch.set_default_dtype(default_dtype)

    # 3) モデルを適切なデータ型に変換

    # True
    else:
        # bfloat16に変換
        model = model.to(torch_dtype)

    # False
    if from_pretrained:
        model = load_checkpoint(
            model,
            from_pretrained,
            cache_dir=cache_dir,
            device_map=device_map,
            strict=strict_load,
        )

    return model

In [ ]:
if True:
    logger.info(f"MMDiTを検証")
    # {'type': 'flux',
    #  'from_pretrained': None,
    #  'strict_load': False,
    #  'guidance_embed': False,
    #  'fused_qkv': False,
    #  'use_liger_rope': True,
    #  'grad_ckpt_settings': (8, 100),
    #  'in_channels': 64,
    #  'vec_in_dim': 768,
    #  'context_in_dim': 4096,
    #  'hidden_size': 384,
    #  'mlp_ratio': 4.0,
    #  'num_heads': 3,
    #  'depth': 1,
    #  'depth_single_blocks': 38,
    #  'axes_dim': [16, 56, 56],
    #  'theta': 10000,
    #  'qkv_bias': True}

    model = Flux(
        from_pretrained=None,
        strict_load=False,
        guidance_embed=False,
        fused_qkv=False,
        use_liger_rope=True,
        grad_ckpt_settings=(8, 100),
        in_channels=64,
        vec_in_dim=768,
        context_in_dim=4096,
        hidden_size=384,
        mlp_ratio=4.0,
        num_heads=3,
        depth=1,
        depth_single_blocks=38,
        axes_dim=[16, 56, 56],
        theta=10000,
        qkv_bias=True,
    )

    # (batch_size, seq_len, in_channels)
    sample_input = torch.randn(1, 16, 64).to("cuda").to(torch.bfloat16)

    # (batch_size, seq_len)
    sample_img_ids = torch.randint(0, 10000, (1, 16, 3)).to("cuda")

    # (batch_size, seq_len, context_in_dim)
    sample_txt = torch.randn(1, 16, 4096).to("cuda").to(torch.bfloat16)

    # (batch_size, seq_len)
    sample_txt_ids = torch.randint(0, 10000, (1, 16, 3)).to("cuda")

    # (batch_size,)
    sample_timesteps = torch.randint(0, 1000, (1,)).to("cuda").to(torch.bfloat16)

    # (batch_size, vec_in_dim)
    sample_y_vec = torch.randn(1, 768).to("cuda").to(torch.bfloat16)

    output = model(
        img=sample_input,
        img_ids=sample_img_ids,
        txt=sample_txt,
        txt_ids=sample_txt_ids,
        timesteps=sample_timesteps,
        y_vec=sample_y_vec,
    )

    # (1, 16, patch_size ** 2 * out_channels)
    logger.info(output.shape)